# Library

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.error import HTTPError

import http.client as httplib  # or http.client if you're on Python 3 # httplib
httplib._MAXHEADERS = 10000

import time
import datetime
from datetime import datetime, timedelta

import re

import json

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import sys

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# pathes
art_project_path = '/content/drive/MyDrive/Art/Art_Project'
art_project_path_backups = '/content/drive/MyDrive/Art/Art_Project/parsing_backups'
art_project_path_tests = '/content/drive/MyDrive/Art/Art_Project/parsing_tests'

# files
page_s = '/page_parsing_status.csv'

## Sites' list for parsing

In [3]:
urls_news = ['https://www.theartnewspaper.ru/', 'https://artinvestment.ru/', 'http://artuzel.com/', 'https://artchive.ru/'] # 'https://artguide.com/', 'http://artuzel.com/', 
# https://artguide.com/ - <div class="chromeframe">
#    Ваш браузер <em>устарел!</em> <a href="http://browsehappy.com/">Установите другой браузер</a> или
#    <a href="http://www.google.com/chromeframe/?redirect=true">Google Chrome Frame</a>, чтобы просматривать этот сайт.
urls_datasets = ['https://artchive.ru/', 'https://artinvestment.ru/', 'http://artuzel.com/']
urls_archives = ['https://artchive.ru/', 'https://russianartarchive.net/ru'] # есть API - https://api.raan.garagemca.org/documentation/#/
urls_fairs = ['https://www.cosmoscow.com/ru/']

# Unused code

In [ ]:
# payload = {'key1': 'value1', 'key2': ['value2', 'value3']}
# r = requests.get('https://httpbin.org/get', params=payload)
# r.json()

In [ ]:
# print(artinvest_page.prettify())

In [ ]:
#                        'def_not_for_pars': {},
#                        'prob_for_pars_articles': {},
#                        'prob_for_pars_ds_other': {}

# Functions

## Data mining
1. __get_page(url)__ -> bs_page - Collect request status

2. Files' functions:
- 2.1 __open_file_csv(file, folder)__ -> df - Open file.csv
- 2.2 __save_df_file(df, file, folder)__ -> - Save df to csv file

In [4]:
# 1. Collect request status - saving every get result

def get_page(url):
  start_time = time.time()

  response = requests.get(url, headers={'User-Agent': 'Chrome/97.0.4692.71'}) # timeout=0.001
# Edge/97.0.1072.55
# Chrome/97.0.4692.71
  try:
    response.raise_for_status()

  except requests.exceptions.HTTPError: # for correction
    sys.exit(print('HTTPError: ' + str(url)))

#except (HTTPError, MissingSchema): 
# 503 Server Error: Service Temporarily Unavailable for url: https://artinvestment.ru/news/artnews/ 
# Invalid URL

  else:
    get_page_stat_new = pd.DataFrame({'page_url': [url],
                                      'status_code': [response.status_code],
                                      'encoding': [response.encoding],
                                      'raise_for_status': [response.raise_for_status()],
                                      'parsing_date': [datetime.now().date()],
                                      'parsing_time': [datetime.now().time()],
                                      'parsing_sec_speed': [time.time() - start_time]})

    bs_page = BeautifulSoup(response.text, 'html.parser')
  
    get_page_stat = open_file_csv(page_s, 'live') # 2. Open file.csv
    get_page_stat = pd.concat([get_page_stat, get_page_stat_new])
    save_df_file(get_page_stat, page_s, 'live') # 3. Save df to csv file

    return bs_page

In [5]:
# 2.1 Open file.csv
def open_file_csv(file, folder): # folder: live, backup or test
  if folder=='live':
    df = pd.read_csv(art_project_path + file)
  elif folder=='backup':
    df = pd.read_csv(art_project_path_backups + file)
  elif folder=='test':
    df = pd.read_csv(art_project_path_tests + file)
  return df

# 2.2 Save df to csv file
def save_df_file(df, file, folder): # folder: live, backup or test
  if folder=='live':
    df.to_csv(art_project_path + file, index=False)
  elif folder=='backup':
    df.to_csv(art_project_path_backups + file, index=False)
  elif folder=='test':
    df.to_csv(art_project_path_tests + file, index=False)

In [6]:
# Date transfomation
def date_transfomation(cell):
  try:
    cell = pd.to_datetime(datetime.strptime(cell, '%d.%m.%Y').strftime('%Y-%m-%d')).date()
  except ValueError:
    cell = pd.to_datetime(cell).date()
  return cell

## Navigation tree
3. __save_parsing_dict(parsing_dict)__ -> - Save dict with sites for parsing:
- 3.1 with sites for parsing
- 3.2 with the last navigation tree
4. __open_parsing_dict()__ -> parsing_dict - Open dict with sites for parsing:
- 4.1 with sites for parsing
- 4.2 with the last navigation tree
5. __check_tree(file, new_tree)__ -> - Checking for updates in navigation tree ***(!!! not for tanp)***
6. __get_add_dict(curr_total_tree, curr_total_title_list, curr_total_links_list)__ -> add_dict - Finding what is not in current total tree dict

In [7]:
current_dict = '/sites_for_parsing.txt'
artinvestment_tree = '/artinvestment_current_tree.txt'
theartnewspaper_tree = '/theartnewspaper_current_tree.txt'
artchive_tree = '/artchive_current_tree.txt'
artuzel_tree = '/artuzel_current_tree.txt'

tanp_total_tree = '/theartnewspaper_total_tree.txt'
artinvest_total_tree = '/artinvestment_total_tree.txt'
artch_total_tree = '/artchive_total_tree.txt'
artuzel_total_tree = '/artuzel_total_tree.txt'

# 3. Save dict:
# 3.1 with sites for parsing
def save_parsing_dict(parsing_dict):
  with open(art_project_path + current_dict, 'w') as outfile:
    json.dump(parsing_dict, outfile)

# 3.2 with the last navigation tree
def save_dict(tree_dict, file):
  with open(art_project_path + file, 'w') as outfile:
    json.dump(tree_dict, outfile)

# 4. Open dict:
# 4.1 with sites for parsing
def open_parsing_dict():
  with open(art_project_path + current_dict, 'rb') as infile:
    parsing_dict = json.load(infile)
  return parsing_dict

# 4.2 with the last navigation tree
def open_dict(file):
  with open(art_project_path + file, 'rb') as infile:
    tree_dict = json.load(infile)
  return tree_dict

In [8]:
# 5. Checking for updates in navigation tree
def check_tree(file, new_tree):
  current_tree = open_dict(file) # 4.2 with the last navigation tree
  in_old_tree_dict = {}
  in_new_tree_dict = {}
  for key in current_tree: # str format
    if (int(key) in new_tree and current_tree[key] != new_tree[int(key)]):
      in_old_tree_dict[int(key)] = current_tree[key] # int format
    elif int(key) not in new_tree:
      in_old_tree_dict[int(key)] = current_tree[key] # int format
  for key in new_tree: # int format
    if (key not in in_old_tree_dict and str(key) in current_tree and new_tree[key] != current_tree[str(key)]):
      in_new_tree_dict[key] = new_tree[key] # int format
    elif (key not in in_old_tree_dict and str(key) not in current_tree):
      in_new_tree_dict[key] = new_tree[key] # int format

  if len(current_tree) == len(new_tree) and len(in_old_tree_dict) == 0 and len(in_new_tree_dict) == 0:
    result = print('Nothing has changed')
  else:
    result = print('Length of navigation three has changed - {}\nChanges in the old tree are: {}\nChanges in the new tree are: {}'.format(len(current_tree)!=len(new_tree), in_old_tree_dict, in_new_tree_dict))

  return result

In [61]:
# 6. Finding what is not in current total tree dict
def get_add_dict(curr_total_tree, curr_total_title_list, curr_total_link_list):
  
  total_title_list_old = []
  for k1, v1 in curr_total_tree.items():
    for k2, v2 in v1.items():
      total_title_list_old.append(v2[1])
  print('total_title_list_old length is {}'.format(len(total_title_list_old)))

  new_not_in_old_list = list(set(curr_total_title_list) - set(total_title_list_old))
  print('count of titles that is not in the current total tree is {}'.format(len(new_not_in_old_list)))

  count_list = []
  count = len(total_title_list_old)
  for t in new_not_in_old_list:
    count += 1
    count_list.append(count)

  add_links_list = []
  for t in new_not_in_old_list:
    if t in curr_total_title_list: # curr_list_titles
      add_links_list.append(curr_total_link_list[curr_total_title_list.index(t)])
    else:
      add_links_list.append('-')

  list_links = []
  for l in add_links_list:
    list_links.append([l, new_not_in_old_list[add_links_list.index(l)]])

  global add_dict
  add_dict = {key: value for key, value in zip(count_list, list_links)}
  return add_dict

# artinvestment.ru
- nav tree contains not only articles' sections, but a lot if extra info

## Navigation tree

In [10]:
# Collect page data
artinvest_url = urls_news[1]
artinvest_page = get_page(artinvest_url) # 1. Collect request status
artinvest_page.title.string

'Цены на картины'

In [11]:
artinvest_tree = artinvest_page.find('nav', id='tree')

### First time

In [ ]:
# 0. Collect topics for parsing FOR THE FIRST TIME

def get_topic_dict(nav_tree):
  list_titles = []
  list_links = []
  count_list = []
  count = 1
  for a in nav_tree.find_all('a'):
    if a.get('title') is not None:
      if str(a.get('title')) not in list_titles:
        list_titles.append(str(a.get('title')))
        count_list.append(count)
        count = count+1
        list_links.append([str(a.get('href')), str(a.get('title'))])
    else:
      if a.get_text() is not None:
        if str(a.get_text()) not in list_titles:
          list_titles.append(str(a.get_text()))
          count_list.append(count)
          count = count+1
          list_links.append([str(a.get('href')), str(a.get_text())])

  tree_dict = {key: value for key, value in zip(count_list, list_links)}
  return tree_dict

In [ ]:
#artinvest_total_tree_dict = get_topic_dict(artinvest_tree) # 0. Collect topics for parsing FOR THE FIRST TIME
#save_dict(artinvest_total_tree_dict, artinvest_total_tree) # 3.2 with the last navigation tree

### All next times

In [12]:
# 0. Collect topics for parsing ALL NEXT TIMES

def get_topic_dict(nav_tree):
  list_titles = []
  list_links = []
  for a in nav_tree.find_all('a'):
    if a.get('title') is not None:
      if str(a.get('title')) not in list_titles:
        list_titles.append(str(a.get('title')))
        list_links.append(str(a.get('href')))
    else:
      if a.get_text() is not None:
        if str(a.get_text()) not in list_titles:
          list_titles.append(str(a.get_text()))
          list_links.append(str(a.get('href')))

  return list_titles, list_links

In [13]:
# Collect topics for parsing from navigation tree
curr_list_titles, curr_list_links = get_topic_dict(artinvest_tree) # 0. Collect topics for parsing ALL NEXT TIMES
print('curr_list_titles length is {}'.format(len(curr_list_titles)))

artinvest_curr_total_tree = open_dict(artinvest_total_tree) # 4.2 with the last navigation tree

get_add_dict(artinvest_curr_total_tree, curr_list_titles, curr_list_links) # 6. Finding what is not in current total tree dict

curr_list_titles length is 48
total_title_list_old length is 48
count of titles that is not in the current total tree is 0


{}

#### total tree dict updating

In [ ]:
prob_for_pars_articles_dict = artinvest_curr_total_tree['prob_for_pars_articles']

for k, v in add_dict.items():
  prob_for_pars_articles_dict.update({str(k): v})
len(prob_for_pars_articles_dict)

19

In [ ]:
# updating the current total tree
artinvest_curr_total_tree.update({'prob_for_pars_articles': prob_for_pars_articles_dict})

artinvest_total_list_old = []
for k1, v1 in artinvest_curr_total_tree.items():
  for k2, v2 in v1.items():
    artinvest_total_list_old.append(v2[1])
len(artinvest_total_list_old)

48

In [ ]:
save_dict(artinvest_curr_total_tree, artinvest_total_tree) # 3.2 with the last navigation tree

### logic of choosing articles category

In [14]:
prob_for_pars_articles_dict = artinvest_curr_total_tree['prob_for_pars_articles']

list_links = []
list_titles = []

for k, v in prob_for_pars_articles_dict.items():
  list_links.append(v[0])
  list_titles.append(v[1])

records_total_list = []
pages_total_list = []
records_max_list = []
records_value_list = []
last_article_date_list = []

for link in list_links: # page with news list of certain category
  page = get_page(link) # 1. Collect request status

  # counter block of the page with news list
  table_blocks = page.find_all('div', class_='pager')

  records_total_list.append(int(table_blocks[0].find_all('em')[0].text))
  pages_total_list.append(int(table_blocks[0].find_all('em')[1].text))
  records_max_list.append(int(table_blocks[0].find('input').get('title').split(' ')[1]))
  records_value_list.append(int(table_blocks[0].find('input').get('value')))

  # current page data block of the page with news list
  list_advices = page.find_all('ul', class_='list advices')

  em = list_advices[0].find_all('em')[0]
  last_article_date_list.append(em.text)

df_counter = pd.DataFrame({'page_url': list_links,
                           'page_title': list_titles,
                           'records_total': records_total_list,
                           'pages_total': pages_total_list,
                           'records_max': records_max_list,
                           'records_value': records_value_list,
                           'last_article_date': last_article_date_list
                           })
df_counter['last_article_date'] = df_counter['last_article_date'].dropna().apply(lambda x: date_transfomation(x).year)

curr_total_title_list = df_counter.loc[df_counter['last_article_date']==datetime.now().date().year]['page_title'].tolist()
curr_total_title_list

['Рынок искусства: аналитика и прогнозы',
 'Инвестиционные идеи на рынке искусства',
 '21-й век',
 'Обзоры событий в мире искусства',
 'Экспертиза, реставрация, хранение',
 'Первые имена',
 'Новости искусства',
 'Аукционы произведений искусства',
 'Выставки произведений искусства']

In [ ]:
#['Рынок искусства: аналитика и прогнозы',
# 'Инвестиционные идеи на рынке искусства',
# '21-й век',
# 'Первые имена',
# 'Новости искусства',
# 'Аукционы произведений искусства',
# 'Выставки произведений искусства']

### Parsing dict updates

In [15]:
# new dict for current parsing
new_tree_dict = {}
for k, v in prob_for_pars_articles_dict.items():
  if v[1] in curr_total_title_list:
    new_tree_dict.update({int(k): prob_for_pars_articles_dict[k]})

new_tree_dict

{21: ['https://artinvestment.ru/invest/analytics/',
  'Рынок искусства: аналитика и прогнозы'],
 22: ['https://artinvestment.ru/invest/ideas/',
  'Инвестиционные идеи на рынке искусства'],
 24: ['https://artinvestment.ru/invest/xxicentury/', '21-й век'],
 28: ['https://artinvestment.ru/invest/events/',
  'Обзоры событий в мире искусства'],
 32: ['https://artinvestment.ru/invest/misc/',
  'Экспертиза, реставрация, хранение'],
 37: ['https://artinvestment.ru/invest/first_names/', 'Первые имена'],
 39: ['https://artinvestment.ru/news/artnews/', 'Новости искусства'],
 40: ['https://artinvestment.ru/news/auctnews/',
  'Аукционы произведений искусства'],
 41: ['https://artinvestment.ru/news/exhibitions/',
  'Выставки произведений искусства']}

In [16]:
check_tree(artinvestment_tree, new_tree_dict)

Length of navigation three has changed - True
Changes in the old tree are: {}
Changes in the new tree are: {28: ['https://artinvestment.ru/invest/events/', 'Обзоры событий в мире искусства'], 32: ['https://artinvestment.ru/invest/misc/', 'Экспертиза, реставрация, хранение']}


In [17]:
index_list = []
for k, v in new_tree_dict.items():
  index_list.append(int(k))

In [18]:
artinvest_datasets = [] # different structure
artinvest_news = [39, 40, 41]
# 'Новости искусства', 'Аукционы произведений искусства', 'Выставки произведений искусства'
artinvest_articles = list(set(index_list) - set(artinvest_news))
artinvest_others = [] # different structure

In [19]:
save_dict(new_tree_dict, artinvestment_tree) # 3.2 with the last navigation tree

## Contacts

In [20]:
contacts_page_url = 'https://artinvestment.ru/contacts/'
contacts_page = get_page(contacts_page_url)
contacts_page.title.string

'Контакты'

In [21]:
contact_tree = contacts_page.find('div', class_='info').find('div', class_='nav')

list_titles, list_links = get_topic_dict(contact_tree)
list_titles

['Условия пользования',
 'Обработка персональных данных',
 'Подписка',
 'Реклама',
 'Telegram',
 'Карта сайта',
 'AI Аукцион',
 'RSS',
 'Галерея Элизиум',
 'Галерея AI "21-й век"']

In [22]:
#contacts_titles = ['Telegram', 'Facebook', 'Instagram']
#contacts_links = []
#for t in contacts_titles:
#  contacts_links.append(list_links[list_titles.index(t)])

In [23]:
contacts_titles = ['Telegram', 'Facebook', 'Instagram']
contacts_links = ['https://t.me/artinvestmentru',
                  'https://www.facebook.com/artinvestment/',
                  'https://www.instagram.com/artinvestment_ru/'
                  ]

## Dict creating

In [24]:
links_and_titles = {'https://artinvestment.ru/': {'news': {'links': [new_tree_dict[x][0] for x in artinvest_news], 
                                                           'titles': [new_tree_dict[x][1] for x in artinvest_news]}, 
                                                  'articles': {'links': [new_tree_dict[x][0] for x in artinvest_articles], 
                                                               'titles': [new_tree_dict[x][1] for x in artinvest_articles]}, 
                                                  'datasets': {'links': [new_tree_dict[x][0] for x in artinvest_datasets], 
                                                               'titles': [new_tree_dict[x][1] for x in artinvest_datasets]}, 
                                                  'others': {'links': [new_tree_dict[x][0] for x in artinvest_others], 
                                                             'titles': [new_tree_dict[x][1] for x in artinvest_others]},
                                                  'contacts': {'links': contacts_links, 
                                                               'titles': contacts_titles}
                                                  }
                     }

links_and_titles

{'https://artinvestment.ru/': {'articles': {'links': ['https://artinvestment.ru/invest/misc/',
    'https://artinvestment.ru/invest/first_names/',
    'https://artinvestment.ru/invest/analytics/',
    'https://artinvestment.ru/invest/ideas/',
    'https://artinvestment.ru/invest/xxicentury/',
    'https://artinvestment.ru/invest/events/'],
   'titles': ['Экспертиза, реставрация, хранение',
    'Первые имена',
    'Рынок искусства: аналитика и прогнозы',
    'Инвестиционные идеи на рынке искусства',
    '21-й век',
    'Обзоры событий в мире искусства']},
  'contacts': {'links': ['https://t.me/artinvestmentru',
    'https://www.facebook.com/artinvestment/',
    'https://www.instagram.com/artinvestment_ru/'],
   'titles': ['Telegram', 'Facebook', 'Instagram']},
  'datasets': {'links': [], 'titles': []},
  'news': {'links': ['https://artinvestment.ru/news/artnews/',
    'https://artinvestment.ru/news/auctnews/',
    'https://artinvestment.ru/news/exhibitions/'],
   'titles': ['Новости иск

In [ ]:
# save_parsing_dict(links_and_titles) # 3.1 Save dict with sites for parsing

## Dict updating

In [25]:
sites_for_parsing = open_parsing_dict() # 4.1 Open dict with sites for parsing
sites_for_parsing.update(links_and_titles)

In [28]:
len(sites_for_parsing)

4

In [27]:
save_parsing_dict(sites_for_parsing) # 3.1 Save dict with sites for parsing

## Художники и цены

In [ ]:
# Художники и цены
url = list_links[0]

start_time = time.time()
response = requests.get(url, headers={'User-Agent': 'Chrome/97.0.4692.71'}) # timeout=0.001
# Edge/97.0.1072.55
# Chrome/97.0.4692.71
for key, value in tree_dict.items():
  if key==url:
    print(value)
print(response.status_code)
print(response.encoding)
print(response.raise_for_status())
print("--- %s seconds ---" % (time.time() - start_time))

Художники и цены
200
UTF-8
None
--- 1.8451573848724365 seconds ---


In [ ]:
artists_page = BeautifulSoup(response.text, 'html.parser')
#print(artists_page.prettify())

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#" xml:lang="ru">
 <head>
  <!-- Global site tag (gtag.js) - Google Analytics -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-66149674-1">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
        gtag('js', new Date());

        gtag('config', 'UA-66149674-1', { 'optimize_id': 'GTM-T4V5FRP'});

        gtag('config', 'AW-474414610');
  </script>
  <!-- Anti-flicker snippet (recommended)  -->
  <style>
   .async-hide { opacity: 0 !important}
  </style>
  <script>
   (function(a,s,y,n,c,h,i,d,e){s.className+=' '+y;h.start=1*new Date;
        h.end=i=function(){s.className=s.className.replace(RegExp(' ?'+y),'')};
        (a[n]=a[n]||[]).hide=h;setTimeout(function(){i();h.end=null},c);h.timeout=c;
    })(window,document.documentElement,'async-hide','dataLayer',4000,
        {'GTM-T4V5FRP':true});
  </script>
  <title>
   Художники и 

In [ ]:
table_blocks = artists_page.find_all('div', class_='pager')
# div class="artists-list"
len(table_blocks)

2

In [ ]:
table_blocks[0]

<div class="pager">
<div class="table"><div>
<div>Записей: <em>14888</em></div>
<div>Страниц: <em>497</em></div>
<div>На стр <input id="page" name="citems" ss="https://artinvestment.ru/auctions/?citems=" title="Max 200" type="text" value="30"/></div>
<div class="pages">
<div class="table"><div>
<div class="first"><a href="https://artinvestment.ru/auctions/?page=1&amp;citems=30" title="Первая">Первая</a></div>
<div class="numbers"><ul>
<li class="current"><a href="https://artinvestment.ru/auctions/">1</a></li>
<li><a href="https://artinvestment.ru/auctions/?page=2&amp;citems=30">2</a></li> <li><a href="https://artinvestment.ru/auctions/?page=3&amp;citems=30">3</a></li> </ul></div>
<div class="next"><a href="https://artinvestment.ru/auctions/?page=2&amp;citems=30">»</a></div>
<div class="last"><a href="https://artinvestment.ru/auctions/?page=497&amp;citems=30" title="Последняя">Последняя</a></div>
</div></div>
</div>
</div></div>
</div>

In [ ]:
artinvest_artists_stat = pd.DataFrame({'page_url': [list_links[0]],
                                       'page_title': [list_titles[0]],
                                       'records_total': [int(table_blocks[0].find_all('em')[0].text)],
                                       'pages_total': [int(table_blocks[0].find_all('em')[1].text)],
                                       'records_max': [int(table_blocks[0].find('input').get('title').split(' ')[1])],
                                       'records_value': [int(table_blocks[0].find('input').get('value'))],
                                       'first_page_url': [table_blocks[0].find('a').get('href')],
                                       'parsing_date': [datetime.now().date()],
                                       'parsing_time': [datetime.now().time()]})
artinvest_artists_stat

,page_url,page_title,records_total,pages_total,records_max,records_value,first_page_url,parsing_date,parsing_time
0,https://artinvestment.ru/auctions/,Художники и цены,14888,497,200,30,https://artinvestment.ru/auctions/?page=1&cite...,2022-01-11,16:56:51.181806


# theartnewspaper.ru
- nav tree contains only articles' sections, nothing else 

## Navigation tree

In [29]:
# Collect page data
theartnewspaper_url = urls_news[0]
theartnewspaper_page = get_page(theartnewspaper_url) # 1. Collect request status
theartnewspaper_page.title.string

'The Art Newspaper Russia — новости искусства'

### First time

In [ ]:
# 0. Collect topics for parsing FOR THE FIRST TIME

def get_topic_dict_tanp(bs_page, url):
  list_titles = []
  list_links = []
  count_list = []
  count = 1
  for mmr in bs_page.find('div', class_='mainMenuRoot').find_all('a'):
    if mmr.get_text() is not None:
      if str(mmr.get_text()) not in list_titles:
        list_titles.append(str(mmr.get_text()))
        count_list.append(count)
        count = count+1
        list_links.append([url[:-1] + str(mmr.get('href')), str(mmr.get_text())])

  for gh in bs_page.find_all('div', class_='groupHeader'):
    for add in gh.find_all('a'):
      if add.get_text() is not None:
        if str(add.get_text()) not in list_titles:
          list_titles.append(str(add.get_text()))
          count_list.append(count)
          count = count+1
          list_links.append([url[:-1] + str(add.get('href')), str(add.get_text())])

  tree_dict = {key: value for key, value in zip(count_list, list_links)}
  return tree_dict

In [ ]:
tanp_total_tree_dict = get_topic_dict_tanp(theartnewspaper_page, theartnewspaper_url)

In [ ]:
tanp_total_tree_dict

{1: ['https://www.theartnewspaper.ru/sections/news/', 'Новости'],
 2: ['https://www.theartnewspaper.ru/sections/shows/', 'Выставки'],
 3: ['https://www.theartnewspaper.ru/sections/markets/', 'Арт-рынок'],
 4: ['https://www.theartnewspaper.ru/sections/conservation/', 'Реставрация'],
 5: ['https://www.theartnewspaper.ru/sections/guerlain/', 'Guerlain'],
 6: ['https://www.theartnewspaper.ru/sections/Premiia-The-Art-Newspaper-Russia/',
  'Премия The Art Newspaper Russia'],
 7: ['https://www.theartnewspaper.ru/sections/luxury/', 'Роскошь'],
 8: ['https://www.theartnewspaper.ru/sections/video/', 'Видео'],
 9: ['https://www.theartnewspaper.ru/sections/gallery/', 'Галереи'],
 10: ['https://www.theartnewspaper.ru/sections/interviu/', 'Интервью']}

In [ ]:
#save_dict(tanp_total_tree_dict, tanp_total_tree) # 3.2 with the last navigation tree

In [ ]:
tanp_curr_total_tree = open_dict(tanp_total_tree) # 4.2 with the last navigation tree
tanp_curr_total_tree

{'1': ['https://www.theartnewspaper.ru/sections/news/', 'Новости'],
 '10': ['https://www.theartnewspaper.ru/sections/spetsproekt/', 'Спецпроект'],
 '11': ['https://www.theartnewspaper.ru/sections/mercedes-benz/',
  'Mercedes-Benz'],
 '12': ['https://www.theartnewspaper.ru/sections/shows/', 'Выставки'],
 '13': ['https://www.theartnewspaper.ru/sections/moskva-rossiia/',
  'Москва Россия'],
 '14': ['https://www.theartnewspaper.ru/sections/london-velikobritaniia/',
  'Лондон Великобритания'],
 '15': ['https://www.theartnewspaper.ru/sections/features/', 'Крупным планом'],
 '16': ['https://www.theartnewspaper.ru/sections/markets/', 'Арт-рынок'],
 '17': ['https://www.theartnewspaper.ru/sections/NFT/', 'NFT'],
 '18': ['https://www.theartnewspaper.ru/sections/Cartier/', 'Cartier'],
 '19': ['https://www.theartnewspaper.ru/sections/biennale/', 'Биеннале'],
 '2': ['https://www.theartnewspaper.ru/sections/rim-italiia/', 'Рим Италия'],
 '20': ['https://www.theartnewspaper.ru/sections/niu-iork-ssha/'

In [ ]:
tanp_curr_total_tree = {'def_not_for_pars': {},
                        'prob_for_pars_articles': {'1': ['https://www.theartnewspaper.ru/sections/news/', 'Новости'],
                                                   '10': ['https://www.theartnewspaper.ru/sections/spetsproekt/', 'Спецпроект'],
                                                   '11': ['https://www.theartnewspaper.ru/sections/mercedes-benz/', 'Mercedes-Benz'],
                                                   '12': ['https://www.theartnewspaper.ru/sections/shows/', 'Выставки'],
                                                   '13': ['https://www.theartnewspaper.ru/sections/moskva-rossiia/', 'Москва Россия'],
                                                   '14': ['https://www.theartnewspaper.ru/sections/london-velikobritaniia/', 'Лондон Великобритания'],
                                                   '15': ['https://www.theartnewspaper.ru/sections/features/', 'Крупным планом'],
                                                   '16': ['https://www.theartnewspaper.ru/sections/markets/', 'Арт-рынок'],
                                                   '17': ['https://www.theartnewspaper.ru/sections/NFT/', 'NFT'],
                                                   '18': ['https://www.theartnewspaper.ru/sections/Cartier/', 'Cartier'],
                                                   '19': ['https://www.theartnewspaper.ru/sections/biennale/', 'Биеннале'],
                                                   '2': ['https://www.theartnewspaper.ru/sections/rim-italiia/', 'Рим Италия'],
                                                   '20': ['https://www.theartnewspaper.ru/sections/niu-iork-ssha/', 'Нью-Йорк США'],
                                                   '21': ['https://www.theartnewspaper.ru/sections/parizh-frantsiia/', 'Париж Франция'],
                                                   '22': ['https://www.theartnewspaper.ru/sections/conservation/', 'Реставрация'],
                                                   '23': ['https://www.theartnewspaper.ru/sections/muzei-budushchego/', 'Музей будущего'],
                                                   '24': ['https://www.theartnewspaper.ru/sections/Premiia-The-Art-Newspaper-Russia/', 'Премия The Art Newspaper Russia'],
                                                   '25': ['https://www.theartnewspaper.ru/sections/sankt-peterburg-rossiia/', 'Санкт-Петербург Россия'],
                                                   '26': ['https://www.theartnewspaper.ru/sections/museums/', 'Музей'],
                                                   '27': ['https://www.theartnewspaper.ru/sections/Tula-i-Tulskaia-oblast-Rossiia/', 'Тула и Тульская область Россия'],
                                                   '28': ['https://www.theartnewspaper.ru/sections/nizhnii-novgorod/', 'Нижний Новгород Россия'],
                                                   '29': ['https://www.theartnewspaper.ru/sections/chikago-ssha/', 'Чикаго США'],
                                                   '3': ['https://www.theartnewspaper.ru/sections/Antverpen-Belgiia/', 'Антверпен Бельгия'],
                                                   '30': ['https://www.theartnewspaper.ru/sections/milan-italiia/', 'Милан Италия'],
                                                   '31': ['https://www.theartnewspaper.ru/sections/mneniia/', 'Мнения'],
                                                   '32': ['https://www.theartnewspaper.ru/sections/leningradskaia-oblast/', 'Ленинградская область Россия'],
                                                   '33': ['https://www.theartnewspaper.ru/sections/velikii-novgorod-rossiia/', 'Великий Новгород Россия'],
                                                   '34': ['https://www.theartnewspaper.ru/sections/velikobritaniia/', 'Великобритания'],
                                                   '35': ['https://www.theartnewspaper.ru/sections/los-andzheles-ssha/', 'Лос-Анджелес США'],
                                                   '36': ['https://www.theartnewspaper.ru/sections/Partnerstvo/', 'Промо'],
                                                   '37': ['https://www.theartnewspaper.ru/sections/priglasheniia/', 'Приглашения'],
                                                   '38': ['https://www.theartnewspaper.ru/sections/books/', 'Книги'],
                                                   '39': ['https://www.theartnewspaper.ru/sections/nekrolog/', 'Некролог'],
                                                   '4': ['https://www.theartnewspaper.ru/sections/ekaterinburg-rossiia/', 'Екатеринбург Россия'],
                                                   '40': ['https://www.theartnewspaper.ru/sections/Audi-Rossiia/', 'Audi Россия'],
                                                   '41': ['https://www.theartnewspaper.ru/sections/kolonka/', 'Колонка'],
                                                   '42': ['https://www.theartnewspaper.ru/sections/puteshestvie/', 'Путешествие'],
                                                   '43': ['https://www.theartnewspaper.ru/sections/stambul-turtsiia/', 'Стамбул Турция'],
                                                   '44': ['https://www.theartnewspaper.ru/sections/rossiia/', 'Россия'],
                                                   '45': ['https://www.theartnewspaper.ru/sections/artists/', 'Художники'],
                                                   '46': ['https://www.theartnewspaper.ru/sections/COVID-19/', 'COVID-19'],
                                                   '47': ['https://www.theartnewspaper.ru/sections/arkhitektura/', 'Архитектура'],
                                                   '48': ['https://www.theartnewspaper.ru/sections/samara-rossiia/', 'Самара Россия'],
                                                   '49': ['https://www.theartnewspaper.ru/sections/kinofestival/', 'Кинофестиваль'],
                                                   '5': ['https://www.theartnewspaper.ru/sections/florentsiia-italiia/', 'Флоренция Италия'],
                                                   '50': ['https://www.theartnewspaper.ru/sections/moskovskaia-oblast/', 'Московская область Россия'],
                                                   '51': ['https://www.theartnewspaper.ru/sections/iarmarki/', 'Ярмарки'],
                                                   '52': ['https://www.theartnewspaper.ru/sections/ssha/', 'США'],
                                                   '53': ['https://www.theartnewspaper.ru/sections/kazan-rossiia/', 'Казань Россия'],
                                                   '54': ['https://www.theartnewspaper.ru/sections/calendar/', 'Календарь'],
                                                   '55': ['https://www.theartnewspaper.ru/sections/guerlain/', 'Guerlain'],
                                                   '56': ['https://www.theartnewspaper.ru/sections/premii-i-nagrady/', 'Премии и награды'],
                                                   '6': ['https://www.theartnewspaper.ru/sections/naznacheniia/', 'Назначения'],
                                                   '7': ['https://www.theartnewspaper.ru/sections/luxury/', 'Роскошь'],
                                                   '8': ['https://www.theartnewspaper.ru/sections/interviu/', 'Интервью'],
                                                   '9': ['https://www.theartnewspaper.ru/sections/festivali/', 'Фестивали']
                                                   },
                        'prob_for_pars_ds_other': {'57': ['https://www.theartnewspaper.ru/sections/gallery/', 'Галереи'],
                                                   '58': ['https://www.theartnewspaper.ru/sections/video/', 'Видео']
                                                   }
                        }

In [ ]:
#save_dict(tanp_curr_total_tree, tanp_total_tree) # 3.2 with the last navigation tree

### All next times

In [30]:
# 0. Collect topics for parsing ALL NEXT TIMES

def get_topic_dict_tanp(bs_page, url):
  list_titles = []
  list_links = []
  for mmr in bs_page.find('div', class_='mainMenuRoot').find_all('a'):
    if mmr.get_text() is not None:
      if str(mmr.get_text()) not in list_titles:
        list_titles.append(str(mmr.get_text()))
        list_links.append(url[:-1] + str(mmr.get('href')))

  for gh in bs_page.find_all('div', class_='groupHeader'):
    for add in gh.find_all('a'):
      if add.get_text() is not None:
        if str(add.get_text()) not in list_titles:
          list_titles.append(str(add.get_text()))
          list_links.append(url[:-1] + str(add.get('href')))

  return list_titles, list_links

In [62]:
# Collect topics for parsing from navigation tree
curr_list_titles, curr_list_links = get_topic_dict_tanp(theartnewspaper_page, theartnewspaper_url) # 0. Collect topics for parsing ALL NEXT TIMES
print('curr_list_titles length is {}'.format(len(curr_list_titles)))

# sections that are not in the navigation tree (don't have links => manual collecting)
links_list = []
page_title_list = []
for prenews in theartnewspaper_page.find_all('div', class_='postPreviewsV2Root js-fix-post-previews'):
  for a in prenews.find_all('a'):
    links_list.append(a.get('href'))
  for p_title in prenews.find_all('div', class_='postPreviewsItemSection'):
    page_title_list.append(p_title.get_text())

add_title_list = list(set(page_title_list) - set(curr_list_titles))

add_link_list = []
for t_idx in add_title_list:
  add_article_url = theartnewspaper_url[:-1] + links_list[page_title_list.index(t_idx)]
  article_page = get_page(add_article_url)
  tag_link = article_page.find('div', class_='postSectionsRoot').find_all('a')[0].get('href')
  add_link_list.append(theartnewspaper_url[:-1] + tag_link)

curr_total_title_list = curr_list_titles + add_title_list
curr_total_link_list = curr_list_links + add_link_list
print('curr_total_title_list length is {}'.format(len(curr_total_title_list)))

tanp_curr_total_tree = open_dict(tanp_total_tree) # 4.2 with the last navigation tree

get_add_dict(tanp_curr_total_tree, curr_total_title_list, curr_total_link_list) # 6. Finding what is not in current total tree dict

curr_list_titles length is 10
curr_total_title_list length is 15
total_title_list_old length is 59
count of titles that is not in the current total tree is 1


{60: ['https://www.theartnewspaper.ru/sections/perm-rossiia/', 'Пермь Россия']}

#### total tree dict updating

In [63]:
# making corrections if it's needed
#add_dict.update({59: ['https://www.theartnewspaper.ru/sections/vashington-ssha/', 'Вашингтон США']})
add_dict

{60: ['https://www.theartnewspaper.ru/sections/perm-rossiia/', 'Пермь Россия']}

In [64]:
prob_for_pars_articles_dict = tanp_curr_total_tree['prob_for_pars_articles']

for k, v in add_dict.items():
  prob_for_pars_articles_dict.update({str(k): v})
len(prob_for_pars_articles_dict)

58

In [65]:
# updating the current total tree
tanp_curr_total_tree.update({'prob_for_pars_articles': prob_for_pars_articles_dict})

tanp_total_list_old = []
for k1, v1 in tanp_curr_total_tree.items():
  for k2, v2 in v1.items():
    tanp_total_list_old.append(v2[1])
len(tanp_total_list_old)

60

In [66]:
# saving total tree dict
save_dict(tanp_curr_total_tree, tanp_total_tree) # 3.2 with the last navigation tree

### Parsing dict updates

In [67]:
# new dict for current parsing
prob_for_pars_articles_dict = tanp_curr_total_tree['prob_for_pars_articles']

new_tree_dict = {}
for k, v in prob_for_pars_articles_dict.items():
  if v[1] in curr_total_title_list:
    new_tree_dict.update({int(k): prob_for_pars_articles_dict[k]})

new_tree_dict

{1: ['https://www.theartnewspaper.ru/sections/news/', 'Новости'],
 7: ['https://www.theartnewspaper.ru/sections/luxury/', 'Роскошь'],
 8: ['https://www.theartnewspaper.ru/sections/interviu/', 'Интервью'],
 12: ['https://www.theartnewspaper.ru/sections/shows/', 'Выставки'],
 13: ['https://www.theartnewspaper.ru/sections/moskva-rossiia/',
  'Москва Россия'],
 16: ['https://www.theartnewspaper.ru/sections/markets/', 'Арт-рынок'],
 22: ['https://www.theartnewspaper.ru/sections/conservation/', 'Реставрация'],
 24: ['https://www.theartnewspaper.ru/sections/Premiia-The-Art-Newspaper-Russia/',
  'Премия The Art Newspaper Russia'],
 26: ['https://www.theartnewspaper.ru/sections/museums/', 'Музей'],
 38: ['https://www.theartnewspaper.ru/sections/books/', 'Книги'],
 42: ['https://www.theartnewspaper.ru/sections/puteshestvie/', 'Путешествие'],
 55: ['https://www.theartnewspaper.ru/sections/guerlain/', 'Guerlain'],
 60: ['https://www.theartnewspaper.ru/sections/perm-rossiia/', 'Пермь Россия']}

In [68]:
check_tree(theartnewspaper_tree, new_tree_dict)

Length of navigation three has changed - True
Changes in the old tree are: {36: ['https://www.theartnewspaper.ru/sections/Partnerstvo/', 'Промо'], 56: ['https://www.theartnewspaper.ru/sections/premii-i-nagrady/', 'Премии и награды'], 59: ['https://www.theartnewspaper.ru/sections/vashington-ssha/', 'Вашингтон США']}
Changes in the new tree are: {42: ['https://www.theartnewspaper.ru/sections/puteshestvie/', 'Путешествие'], 60: ['https://www.theartnewspaper.ru/sections/perm-rossiia/', 'Пермь Россия']}


In [69]:
index_list = []
for k, v in new_tree_dict.items():
  index_list.append(int(k))

In [70]:
# 9 and 10 are excluded
theartnewspaper_datasets = []  # different structure
theartnewspaper_news = [1, 12, 16]
# 'Новости', 'Выставки', 'Арт-рынок'
theartnewspaper_others = []  # different structure
# 'Галереи', 'Видео'
theartnewspaper_articles = list(set(index_list) - set(theartnewspaper_news))

In [71]:
# saving current tree dict
save_dict(new_tree_dict, theartnewspaper_tree) # 3.2 with the last navigation tree

## Contacts

In [ ]:
contacts = theartnewspaper_page.find('div', class_='preFooterRoot').find('div', class_='leftCol')
list_links = []
for a in contacts.find_all('a'):
  list_links.append(a.get('href'))

list_links

['https://vk.com/theartnewspaper.russia',
 'https://t.me/TANR_official',
 'https://ok.ru/theartnewspaperrussia/',
 'https://twitter.com/TANRussia',
 'https://www.youtube.com/c/TheArtNewspaperRussia']

In [ ]:
index_list = [3, 0, 1]
contacts_titles = ['Telegram', 'Facebook', 'Instagram']
contacts_links = []
for i in index_list:
  contacts_links.append(list_links[i])

In [72]:
contacts_titles = ['Telegram', 'Facebook', 'Instagram']
contacts_links = ['https://t.me/TANR_official', 
                  'https://www.facebook.com/TheArtNewspaperRussia', 
                  'https://www.instagram.com/theartnewspaperrussia/']

## Dict creating

In [73]:
links_and_titles = {'https://www.theartnewspaper.ru/': {'news': {'links': [new_tree_dict[x][0] for x in theartnewspaper_news], 
                                                                 'titles': [new_tree_dict[x][1] for x in theartnewspaper_news]}, 
                                                        'articles': {'links': [new_tree_dict[x][0] for x in theartnewspaper_articles], 
                                                                     'titles': [new_tree_dict[x][1] for x in theartnewspaper_articles]},
                                                        'datasets': {'links': [new_tree_dict[x][0] for x in theartnewspaper_datasets], 
                                                                     'titles': [new_tree_dict[x][1] for x in theartnewspaper_datasets]}, 
                                                        'others': {'links': [new_tree_dict[x][0] for x in theartnewspaper_others], 
                                                                   'titles': [new_tree_dict[x][1] for x in theartnewspaper_others]},
                                                        'contacts': {'links': contacts_links,
                                                                     'titles': contacts_titles}
                                                        }
                         }
links_and_titles

{'https://www.theartnewspaper.ru/': {'articles': {'links': ['https://www.theartnewspaper.ru/sections/books/',
    'https://www.theartnewspaper.ru/sections/luxury/',
    'https://www.theartnewspaper.ru/sections/interviu/',
    'https://www.theartnewspaper.ru/sections/puteshestvie/',
    'https://www.theartnewspaper.ru/sections/moskva-rossiia/',
    'https://www.theartnewspaper.ru/sections/conservation/',
    'https://www.theartnewspaper.ru/sections/guerlain/',
    'https://www.theartnewspaper.ru/sections/Premiia-The-Art-Newspaper-Russia/',
    'https://www.theartnewspaper.ru/sections/museums/',
    'https://www.theartnewspaper.ru/sections/perm-rossiia/'],
   'titles': ['Книги',
    'Роскошь',
    'Интервью',
    'Путешествие',
    'Москва Россия',
    'Реставрация',
    'Guerlain',
    'Премия The Art Newspaper Russia',
    'Музей',
    'Пермь Россия']},
  'contacts': {'links': ['https://t.me/TANR_official',
    'https://www.facebook.com/TheArtNewspaperRussia',
    'https://www.instagra

## Dict updating

In [74]:
sites_for_parsing = open_parsing_dict() # 4.1 Open dict with sites for parsing
sites_for_parsing.update(links_and_titles)

In [75]:
len(sites_for_parsing)

4

In [76]:
sites_for_parsing

{'https://artchive.ru/': {'articles': {'links': ['https://artchive.ru/publications',
    'https://artchive.ru/publications/analysis',
    'https://artchive.ru/publications/for_beginers',
    'https://artchive.ru/publications/experts',
    'https://artchive.ru/publications/collectors',
    'https://artchive.ru/publications/aritist_biographies',
    'https://artchive.ru/publications/for_artists'],
   'titles': ['Все статьи',
    'Аналитика',
    'Новичкам',
    'Экспертам',
    'Коллекционерам',
    'Персоналии',
    'Художникам']},
  'contacts': {'links': ['https://telegram.me/artchiveru',
    'https://www.facebook.com/artchive.ru',
    'https://www.instagram.com/artchiveru/'],
   'titles': ['Telegram', 'Facebook', 'Instagram']},
  'datasets': {'links': [], 'titles': []},
  'news': {'links': ['https://artchive.ru/news',
    'https://artchive.ru/news/rubric:events',
    'https://artchive.ru/news/rubric:news_about_museums_and_galleries',
    'https://artchive.ru/news/rubric:sales_and_auct

In [77]:
save_parsing_dict(sites_for_parsing) # 3.1 Save dict with sites for parsing

# artchive.ru
- nav tree contains not only articles' sections, but a lot if extra info

## Navigation tree

In [78]:
# Collect page data
artchive_url = urls_news[3]
artchive_page = get_page(artchive_url) # 1. Collect request status
artchive_page.title.string

'Артхив - Социальная сеть художников и ценителей искусств'

### First time

In [ ]:
# 0.1 Collect topics for parsing FOR THE FIRST TIME

def get_topic_dict_artch(bs_page, url):
  list_titles = []
  list_links = []
  count_list = []
  count = 1
  for ul in bs_page.find('ul', class_='header-menu-list c_a_header').find_all('a'):
    if ul.get_text() is not None:
      if str(ul.get_text()) not in list_titles:
        list_titles.append(str(ul.get_text()))
        count_list.append(count)
        count = count+1
        list_links.append([url[:-1] + str(ul.get('href')), str(ul.get_text())])

  tree_dict = {key: value for key, value in zip(count_list, list_links)}
  return tree_dict

In [ ]:
artch_total_tree_dict = get_topic_dict_artch(artchive_page, artchive_url) # 0.1 Collect topics for parsing FOR THE FIRST TIME
artch_total_tree_dict

{1: ['https://artchive.ru/artists', 'Художники'],
 2: ['https://artchive.ru/artworks', 'Картины'],
 3: ['https://artchive.ru/galleries', 'Галереи'],
 4: ['https://artchive.ru/exhibitions', 'Выставки'],
 5: ['https://artchive.ru/news', 'Новости'],
 6: ['https://artchive.ru/quizzes', 'Тесты'],
 7: ['https://artchive.ru/publications', 'Знания'],
 8: ['https://artchive.ru/encyclopedia', 'Энциклопедия'],
 9: ['https://artchive.ru/collections', 'Коллекции'],
 10: ['https://artchive.ru/selections', 'Подборки'],
 11: ['https://artchive.ru/catalog', 'Справочник'],
 12: ['https://artchive.ru/audience/artists', 'Художникам'],
 13: ['https://artchive.ru/audience/dealers', 'Галереям'],
 14: ['https://artchive.ru/audience/collectors', 'Коллекционерам'],
 15: ['https://artchive.ru/audience/artlovers', 'Любителям искусства'],
 16: ['https://artchive.ru/about', 'О проекте'],
 17: ['https://artchive.ru/groups/3', 'Блог Артхива'],
 18: ['https://artchive.ru/about/vacancies', 'Вакансии'],
 19: ['https://a

In [ ]:
# 0.2 Collect sub topics for parsing FOR THE FIRST TIME

def get_sub_topic_dict_artch(bs_page, url, count):
  list_titles = []
  list_links = []
  count_list = []
  for ul in bs_page.find('div', class_='scroll-menu').find_all('a'):
    if ul.get_text() is not None:
      if str(ul.get_text()) not in list_titles:
        list_titles.append(str(ul.get_text()))
        count_list.append(count)
        count = count+1
        list_links.append([url[:-1] + str(ul.get('href')), str(ul.get_text())])

  tree_dict = {key: value for key, value in zip(count_list, list_links)}
  return tree_dict

In [ ]:
news_and_articles = ['Новости', 'Знания']
sub_tree_dict = {}
count = len(artch_total_tree_dict)+1
for k, v in artch_total_tree_dict.items():
  if v[1] in news_and_articles:
    v_page = get_page(v[0]) # 1. Collect request status
    sub_tree_dict.update(get_sub_topic_dict_artch(v_page, artchive_url, count)) # 0.2 Collect sub topics for parsing FOR THE FIRST TIME
    count += len(sub_tree_dict)

In [ ]:
sub_tree_dict

{23: ['https://artchive.ru/news', 'Все новости'],
 24: ['https://artchive.ru/news/rubric:events', 'События и выставки'],
 25: ['https://artchive.ru/news/rubric:news_about_museums_and_galleries',
  'Новости музеев и галерей'],
 26: ['https://artchive.ru/news/rubric:sales_and_auctions', 'Продажи и торги'],
 27: ['https://artchive.ru/news/rubric:modern', 'Современное искусство'],
 28: ['https://artchive.ru/publications', 'Все статьи'],
 29: ['https://artchive.ru/publications/analysis', 'Аналитика'],
 30: ['https://artchive.ru/publications/for_beginers', 'Новичкам'],
 31: ['https://artchive.ru/publications/experts', 'Экспертам'],
 32: ['https://artchive.ru/publications/collectors', 'Коллекционерам'],
 33: ['https://artchive.ru/publications/aritist_biographies', 'Персоналии'],
 34: ['https://artchive.ru/publications/for_artists', 'Художникам']}

In [ ]:
artch_total_tree_dict = {'prob_for_pars_ds_other': {1: ['https://artchive.ru/artists', 'Художники'],
                                                    2: ['https://artchive.ru/artworks', 'Картины'],
                                                    3: ['https://artchive.ru/galleries', 'Галереи'],
                                                    4: ['https://artchive.ru/exhibitions', 'Выставки'],
                                                    8: ['https://artchive.ru/encyclopedia', 'Энциклопедия'],
                                                    9: ['https://artchive.ru/collections', 'Коллекции'],
                                                    10: ['https://artchive.ru/selections', 'Подборки'],
                                                    11: ['https://artchive.ru/catalog', 'Справочник']
                                                    },
                         'prob_for_pars_articles': {23: ['https://artchive.ru/news', 'Все новости'],
                                                    24: ['https://artchive.ru/news/rubric:events', 'События и выставки'],
                                                    25: ['https://artchive.ru/news/rubric:news_about_museums_and_galleries', 'Новости музеев и галерей'],
                                                    26: ['https://artchive.ru/news/rubric:sales_and_auctions', 'Продажи и торги'],
                                                    27: ['https://artchive.ru/news/rubric:modern', 'Современное искусство'],
                                                    28: ['https://artchive.ru/publications', 'Все статьи'],
                                                    29: ['https://artchive.ru/publications/analysis', 'Аналитика'],
                                                    30: ['https://artchive.ru/publications/for_beginers', 'Новичкам'],
                                                    31: ['https://artchive.ru/publications/experts', 'Экспертам'],
                                                    32: ['https://artchive.ru/publications/collectors', 'Коллекционерам'],
                                                    33: ['https://artchive.ru/publications/aritist_biographies', 'Персоналии'],
                                                    34: ['https://artchive.ru/publications/for_artists', 'Художникам']
                                                    },
                         'def_not_for_pars': {5: ['https://artchive.ru/news', 'Новости'],
                                              6: ['https://artchive.ru/quizzes', 'Тесты'],
                                              7: ['https://artchive.ru/publications', 'Знания'],
                                              12: ['https://artchive.ru/audience/artists', 'Художникам'],
                                              13: ['https://artchive.ru/audience/dealers', 'Галереям'],
                                              14: ['https://artchive.ru/audience/collectors', 'Коллекционерам'],
                                              15: ['https://artchive.ru/audience/artlovers', 'Любителям искусства'],
                                              16: ['https://artchive.ru/about', 'О проекте'],
                                              17: ['https://artchive.ru/groups/3', 'Блог Артхива'],
                                              18: ['https://artchive.ru/about/vacancies', 'Вакансии'],
                                              19: ['https://artchive.ru/ad', 'Реклама'],
                                              20: ['https://artchive.ru/sites', 'Создать сайт художника или галереи'],
                                              21: ['https://artchive.ru/pro', 'PRO аккаунт'],
                                              22: ['https://artchive.ru/apps', 'Мобильные приложения']
                                              }
                         }

In [ ]:
#save_dict(artch_total_tree_dict, artch_total_tree) # 3.2 with the last navigation tree

In [ ]:
curr_tree_dict = artch_total_tree_dict['prob_for_pars_articles']
curr_tree_dict

{23: ['https://artchive.ru/news', 'Все новости'],
 24: ['https://artchive.ru/news/rubric:events', 'События и выставки'],
 25: ['https://artchive.ru/news/rubric:news_about_museums_and_galleries',
  'Новости музеев и галерей'],
 26: ['https://artchive.ru/news/rubric:sales_and_auctions', 'Продажи и торги'],
 27: ['https://artchive.ru/news/rubric:modern', 'Современное искусство'],
 28: ['https://artchive.ru/publications', 'Все статьи'],
 29: ['https://artchive.ru/publications/analysis', 'Аналитика'],
 30: ['https://artchive.ru/publications/for_beginers', 'Новичкам'],
 31: ['https://artchive.ru/publications/experts', 'Экспертам'],
 32: ['https://artchive.ru/publications/collectors', 'Коллекционерам'],
 33: ['https://artchive.ru/publications/aritist_biographies', 'Персоналии'],
 34: ['https://artchive.ru/publications/for_artists', 'Художникам']}

In [ ]:
# saving current tree dict
save_dict(curr_tree_dict, artchive_tree) # 3.2 with the last navigation tree

### All next times

In [79]:
# 0.1 Collect topics for parsing ALL NEXT TIMES

def get_topic_dict_artch(bs_page, url):
  list_titles = []
  list_links = []
  for ul in bs_page.find('ul', class_='header-menu-list c_a_header').find_all('a'):
    if ul.get_text() is not None:
      if str(ul.get_text()) not in list_titles:
        list_titles.append(str(ul.get_text()))
        list_links.append(url[:-1] + str(ul.get('href')))

  return list_titles, list_links

In [80]:
# 0.2 Collect sub topics for parsing ALL NEXT TIMES

def get_sub_topic_dict_artch(bs_page, url):
  list_titles = []
  list_links = []
  for ul in bs_page.find('div', class_='scroll-menu').find_all('a'):
    if ul.get_text() is not None:
      if str(ul.get_text()) not in list_titles:
        list_titles.append(str(ul.get_text()))
        list_links.append(url[:-1] + str(ul.get('href')))

  return list_titles, list_links

In [81]:
# Collect topics for parsing from navigation tree
curr_list_titles, curr_list_links = get_topic_dict_artch(artchive_page, artchive_url) # 0.1 Collect topics for parsing ALL NEXT TIMES
print('curr_list_titles length is {}'.format(len(curr_list_titles)))

news_and_articles = ['Новости', 'Знания']

add_title_list = []
add_link_list = []
for t in curr_list_titles:
  if t in news_and_articles:
    t_url = curr_list_links[curr_list_titles.index(t)]
    t_page = get_page(t_url) # 1. Collect request status
    sub_list_titles, sub_list_links = get_sub_topic_dict_artch(t_page, t_url) # 0.2 Collect sub topics for parsing ALL NEXT TIMES
    add_title_list += sub_list_titles
    add_link_list += sub_list_links

curr_total_title_list = curr_list_titles + add_title_list
curr_total_links_list = curr_list_links + add_link_list
print('curr_total_title_list length is {}'.format(len(curr_total_title_list)))

artch_curr_total_tree = open_dict(artch_total_tree) # 4.2 with the last navigation tree

get_add_dict(artch_curr_total_tree, curr_total_title_list, curr_total_links_list) # 6. Finding what is not in current total tree dict

curr_list_titles length is 22
curr_total_title_list length is 34
total_title_list_old length is 34
count of titles that is not in the current total tree is 0


{}

#### total tree dict updating

In [ ]:
prob_for_pars_articles_dict = artch_curr_total_tree['prob_for_pars_articles']

for k, v in add_dict.items():
  prob_for_pars_articles_dict.update({str(k): v})
len(prob_for_pars_articles_dict)

In [ ]:
# updating the current total tree
artch_curr_total_tree.update({'prob_for_pars_articles': prob_for_pars_articles_dict})

artch_total_list_old = []
for k1, v1 in artch_curr_total_tree.items():
  for k2, v2 in v1.items():
    artch_total_list_old.append(v2[1])
len(artch_total_list_old)

In [ ]:
save_dict(artch_curr_total_tree, artch_total_tree) # 3.2 with the last navigation tree

### Parsing dict updates

In [ ]:
prob_for_pars_articles_dict = artch_curr_total_tree['prob_for_pars_articles']

# new dict for current parsing
new_tree_dict = {}
for k, v in prob_for_pars_articles_dict.items():
  if v[1] in curr_total_title_list:
    new_tree_dict.update({int(k): prob_for_pars_articles_dict[k]})

new_tree_dict

{23: ['https://artchive.ru/news', 'Все новости'],
 24: ['https://artchive.ru/news/rubric:events', 'События и выставки'],
 25: ['https://artchive.ru/news/rubric:news_about_museums_and_galleries',
  'Новости музеев и галерей'],
 26: ['https://artchive.ru/news/rubric:sales_and_auctions', 'Продажи и торги'],
 27: ['https://artchive.ru/news/rubric:modern', 'Современное искусство'],
 28: ['https://artchive.ru/publications', 'Все статьи'],
 29: ['https://artchive.ru/publications/analysis', 'Аналитика'],
 30: ['https://artchive.ru/publications/for_beginers', 'Новичкам'],
 31: ['https://artchive.ru/publications/experts', 'Экспертам'],
 32: ['https://artchive.ru/publications/collectors', 'Коллекционерам'],
 33: ['https://artchive.ru/publications/aritist_biographies', 'Персоналии'],
 34: ['https://artchive.ru/publications/for_artists', 'Художникам']}

In [ ]:
check_tree(artchive_tree, new_tree_dict)

Nothing has changed


In [ ]:
#index_list = []
#for k, v in new_tree_dict.items():
#  index_list.append(int(k))

In [ ]:
artchive_datasets = []  # different structure for API
# 'Справочник'
artchive_news = [23, 24, 25, 26, 27]
# 'Все новости', 'События и выставки', 'Новости музеев и галерей', 'Продажи и торги', 'Современное искусство'
artchive_articles = [28, 29, 30, 31, 32, 33, 34]
# 'Все статьи', 'Аналитика', 'Новичкам', 'Экспертам', 'Коллекционерам', 'Персоналии', 'Художникам'
artchive_others = []  # different structure

#artchive_articles = list(set(index_list) - set(artchive_news))

In [ ]:
# saving current tree dict
save_dict(new_tree_dict, artchive_tree) # 3.2 with the last navigation tree

## Contacts

In [ ]:
list_links = []
contacts_1 = artchive_page.find('div', class_='socials-rounded')
for a in contacts_1.find_all('a'):
  list_links.append(a.get('href'))
contacts_2 = artchive_page.find_all('ul', class_='ref-footer-menu-list')
for a in contacts_2[3].find_all('a'):
  list_links.append(a.get('href'))

list_links

['https://www.facebook.com/artchive.ru',
 'https://twitter.com/artchiveru',
 'https://www.instagram.com/artchiveru/',
 'https://vk.com/artchive',
 'https://ok.ru/group/51808152256645',
 'https://www.youtube.com/channel/UC5mnWWD_aR7BM61nJRsapNQ',
 'https://medium.com/@artchive',
 'https://telegram.me/artchiveru',
 'https://www.instagram.com/artchiveru/']

In [ ]:
index_list = [7, 0, 2]
contacts_titles = ['Telegram', 'Facebook', 'Instagram']
contacts_links = []
for i in index_list:
  contacts_links.append(list_links[i])

## Dict creating

In [ ]:
links_and_titles = {'https://artchive.ru/': {'news': {'links': [new_tree_dict[x][0] for x in artchive_news], 
                                                                 'titles': [new_tree_dict[x][1] for x in artchive_news]}, 
                                                        'articles': {'links': [new_tree_dict[x][0] for x in artchive_articles], 
                                                                     'titles': [new_tree_dict[x][1] for x in artchive_articles]},
                                                        'datasets': {'links': [new_tree_dict[x][0] for x in artchive_datasets], 
                                                                     'titles': [new_tree_dict[x][1] for x in artchive_datasets]}, 
                                                        'others': {'links': [new_tree_dict[x][0] for x in artchive_others], 
                                                                   'titles': [new_tree_dict[x][1] for x in artchive_others]},
                                                        'contacts': {'links': contacts_links,
                                                                     'titles': contacts_titles}
                                                        }
                         }
links_and_titles

{'https://artchive.ru/': {'articles': {'links': ['https://artchive.ru/publications',
    'https://artchive.ru/publications/analysis',
    'https://artchive.ru/publications/for_beginers',
    'https://artchive.ru/publications/experts',
    'https://artchive.ru/publications/collectors',
    'https://artchive.ru/publications/aritist_biographies',
    'https://artchive.ru/publications/for_artists'],
   'titles': ['Все статьи',
    'Аналитика',
    'Новичкам',
    'Экспертам',
    'Коллекционерам',
    'Персоналии',
    'Художникам']},
  'contacts': {'links': ['https://telegram.me/artchiveru',
    'https://www.facebook.com/artchive.ru',
    'https://www.instagram.com/artchiveru/'],
   'titles': ['Telegram', 'Facebook', 'Instagram']},
  'datasets': {'links': [], 'titles': []},
  'news': {'links': ['https://artchive.ru/news',
    'https://artchive.ru/news/rubric:events',
    'https://artchive.ru/news/rubric:news_about_museums_and_galleries',
    'https://artchive.ru/news/rubric:sales_and_auct

## Dict updating

In [ ]:
sites_for_parsing = open_parsing_dict() # 4.1 Open dict with sites for parsing
sites_for_parsing.update(links_and_titles)

In [ ]:
len(sites_for_parsing)

3

In [ ]:
sites_for_parsing

{'https://artchive.ru/': {'articles': {'links': ['https://artchive.ru/publications',
    'https://artchive.ru/publications/analysis',
    'https://artchive.ru/publications/for_beginers',
    'https://artchive.ru/publications/experts',
    'https://artchive.ru/publications/collectors',
    'https://artchive.ru/publications/aritist_biographies',
    'https://artchive.ru/publications/for_artists'],
   'titles': ['Все статьи',
    'Аналитика',
    'Новичкам',
    'Экспертам',
    'Коллекционерам',
    'Персоналии',
    'Художникам']},
  'contacts': {'links': ['https://telegram.me/artchiveru',
    'https://www.facebook.com/artchive.ru',
    'https://www.instagram.com/artchiveru/'],
   'titles': ['Telegram', 'Facebook', 'Instagram']},
  'datasets': {'links': [], 'titles': []},
  'news': {'links': ['https://artchive.ru/news',
    'https://artchive.ru/news/rubric:events',
    'https://artchive.ru/news/rubric:news_about_museums_and_galleries',
    'https://artchive.ru/news/rubric:sales_and_auct

In [ ]:
save_parsing_dict(sites_for_parsing) # 3.1 Save dict with sites for parsing

# artuzel.com
- nav tree contains not only articles' sections, but a lot if extra info

## Navigation tree

In [82]:
# Collect page data
artuzel_url = urls_news[2]
artuzel_page = get_page(artuzel_url) # 1. Collect request status
artuzel_page.title.string

'ART Узел | Независимый медиа-проект о современном искусстве'

### First time

In [ ]:
# 0.1 Collect topics for parsing FOR THE FIRST TIME

def get_topic_dict_artuz(bs_page, url):
  list_titles = []
  list_links = []
  count_list = []
  count = 1
  for li in bs_page.find_all('div', class_='content')[2].find_all('li'):
    try:
      if li.find('a').get('href') is not None and str(li.find('a').get_text()) not in list_titles:
        list_titles.append(str(li.find('a').get_text()))
        count_list.append(count)
        count = count+1
        list_links.append([url[:-1] + str(li.find('a').get('href')), str(li.find('a').get_text())])
    except: continue

  tree_dict = {key: value for key, value in zip(count_list, list_links)}
  return tree_dict

In [ ]:
artuz_total_tree_dict = get_topic_dict_artuz(artuzel_page, artuzel_url) # 0.1 Collect topics for parsing FOR THE FIRST TIME
artuz_total_tree_dict

{1: ['http://artuzel.com/competitions', 'Конкурсы и гранты'],
 2: ['http://artuzel.com/news', 'Новости'],
 3: ['http://artuzel.com/exhibitions', 'Выставки'],
 4: ['http://artuzel.com/lectures', 'Лекции'],
 5: ['http://artuzel.comhttp://artuzel.com/ru/regions', 'Не столицы'],
 6: ['http://artuzel.com/interview', 'Интервью и дискуссии'],
 7: ['http://artuzel.com/articles', 'Статьи'],
 8: ['http://artuzel.com/history', 'История'],
 9: ['http://artuzel.com/artists', 'Художники'],
 10: ['http://artuzel.com/dictionary', 'Словарь'],
 11: ['http://artuzel.com/literature', 'Литература'],
 12: ['http://artuzel.com/arhive', 'Архив лекций'],
 13: ['http://artuzel.com/auctions', 'Аукционы, online-галереи'],
 14: ['http://artuzel.com/fair', 'Ярмарки'],
 15: ['http://artuzel.com/festivals', 'Фестивали'],
 16: ['http://artuzel.com/education', 'Образование'],
 17: ['http://artuzel.com/addresses', 'ГАЛЕРЕИ']}

In [ ]:
artuz_total_tree_dict = {'prob_for_pars_ds_other': { 8: ['http://artuzel.com/history', 'История'],
                                                    9: ['http://artuzel.com/artists', 'Художники'],
                                                    10: ['http://artuzel.com/dictionary', 'Словарь'],
                                                    12: ['http://artuzel.com/arhive', 'Архив лекций'],
                                                    13: ['http://artuzel.com/auctions', 'Аукционы, online-галереи'],
                                                    14: ['http://artuzel.com/fair', 'Ярмарки'],
                                                    16: ['http://artuzel.com/education', 'Образование']
                                                    },
                         'prob_for_pars_articles': {1: ['http://artuzel.com/competitions', 'Конкурсы и гранты'],
                                                    2: ['http://artuzel.com/news', 'Новости'],
                                                    3: ['http://artuzel.com/exhibitions', 'Выставки'],
                                                    4: ['http://artuzel.com/lectures', 'Лекции'],
                                                    5: ['http://artuzel.com/ru/regions', 'Не столицы'],
                                                    6: ['http://artuzel.com/interview', 'Интервью и дискуссии'],
                                                    7: ['http://artuzel.com/articles', 'Статьи'],
                                                    15: ['http://artuzel.com/festivals', 'Фестивали']
                                                    },
                         'def_not_for_pars': {11: ['http://artuzel.com/literature', 'Литература'],
                                              17: ['http://artuzel.com/addresses', 'ГАЛЕРЕИ']
                                              }
                         }

In [ ]:
save_dict(artuz_total_tree_dict, artuzel_total_tree) # 3.2 with the last navigation tree

In [ ]:
curr_tree_dict = artuz_total_tree_dict['prob_for_pars_articles']
curr_tree_dict

{1: ['http://artuzel.com/competitions', 'Конкурсы и гранты'],
 2: ['http://artuzel.com/news', 'Новости'],
 3: ['http://artuzel.com/exhibitions', 'Выставки'],
 4: ['http://artuzel.com/lectures', 'Лекции'],
 5: ['http://artuzel.com/ru/regions', 'Не столицы'],
 6: ['http://artuzel.com/interview', 'Интервью и дискуссии'],
 7: ['http://artuzel.com/articles', 'Статьи'],
 15: ['http://artuzel.com/festivals', 'Фестивали']}

In [ ]:
# saving current tree dict
save_dict(curr_tree_dict, artuzel_tree) # 3.2 with the last navigation tree

### All next times

In [83]:
# 0.1 Collect topics for parsing ALL NEXT TIMES

def get_topic_dict_artuz(bs_page, url):
  list_titles = []
  list_links = []
  for li in bs_page.find_all('div', class_='content')[2].find_all('li'):
    try:
      if li.find('a').get('href') is not None and str(li.find('a').get_text()) not in list_titles:
        list_titles.append(str(li.find('a').get_text()))
        list_links.append(url[:-1] + str(li.find('a').get('href')))
    except: continue

  return list_titles, list_links

In [84]:
# Collect topics for parsing from navigation tree
curr_list_titles, curr_list_links = get_topic_dict_artuz(artuzel_page, artuzel_url) # 0.1 Collect topics for parsing ALL NEXT TIMES
print('curr_list_titles length is {}'.format(len(curr_list_titles)))

artuz_curr_total_tree = open_dict(artuzel_total_tree) # 4.2 with the last navigation tree

get_add_dict(artuz_curr_total_tree, curr_list_titles, curr_list_links) # 6. Finding what is not in current total tree dict

curr_list_titles length is 17
total_title_list_old length is 17
count of titles that is not in the current total tree is 0


{}

#### total tree dict updating

In [ ]:
prob_for_pars_articles_dict = artuz_curr_total_tree['prob_for_pars_articles']

for k, v in add_dict.items():
  prob_for_pars_articles_dict.update({str(k): v})
len(prob_for_pars_articles_dict)

In [ ]:
# updating the current total tree
artuz_curr_total_tree.update({'prob_for_pars_articles': prob_for_pars_articles_dict})

artuz_total_list_old = []
for k1, v1 in artuz_curr_total_tree.items():
  for k2, v2 in v1.items():
    artuz_total_list_old.append(v2[1])
len(artuz_total_list_old)

In [ ]:
save_dict(artuz_curr_total_tree, artuzel_total_tree) # 3.2 with the last navigation tree

### Parsing dict updates

In [ ]:
prob_for_pars_articles_dict = artuz_curr_total_tree['prob_for_pars_articles']

# new dict for current parsing
new_tree_dict = {}
for k, v in prob_for_pars_articles_dict.items():
  if v[1] in curr_list_titles:
    new_tree_dict.update({int(k): prob_for_pars_articles_dict[k]})

new_tree_dict

{1: ['http://artuzel.com/competitions', 'Конкурсы и гранты'],
 2: ['http://artuzel.com/news', 'Новости'],
 3: ['http://artuzel.com/exhibitions', 'Выставки'],
 4: ['http://artuzel.com/lectures', 'Лекции'],
 5: ['http://artuzel.com/ru/regions', 'Не столицы'],
 6: ['http://artuzel.com/interview', 'Интервью и дискуссии'],
 7: ['http://artuzel.com/articles', 'Статьи'],
 15: ['http://artuzel.com/festivals', 'Фестивали']}

In [ ]:
check_tree(artuzel_tree, new_tree_dict)

Nothing has changed


In [ ]:
#index_list = []
#for k, v in new_tree_dict.items():
#  index_list.append(int(k))

In [ ]:
artuzel_datasets = []  # different structure for API
artuzel_news = [2, 3, 5]
# 'Новости', 'Выставки', 'Не столицы'
artuzel_articles = [1, 4, 6, 7, 15]
# 'Конкурсы и гранты', 'Лекции', 'Интервью и дискуссии', 'Статьи', 'Фестивали'
artuzel_others = []  # different structure

#artuzel_articles = list(set(index_list) - set(artuzel_news))

In [ ]:
# saving current tree dict
save_dict(new_tree_dict, artchive_tree) # 3.2 with the last navigation tree

## Contacts

In [ ]:
list_links = []
contacts = artuzel_page.find('div', class_='content')
for a in contacts.find_all('a'):
  list_links.append(a.get('href'))

list_links

[]

In [ ]:
# data has blocked and actually doesn't contane required information
contacts_titles = ['Telegram', 'Facebook', 'Instagram']
contacts_links = ['', '', '']

## Dict creating

In [ ]:
links_and_titles = {'https://artuzel.com/': {'news': {'links': [new_tree_dict[x][0] for x in artuzel_news], 
                                                      'titles': [new_tree_dict[x][1] for x in artuzel_news]}, 
                                             'articles': {'links': [new_tree_dict[x][0] for x in artuzel_articles], 
                                                          'titles': [new_tree_dict[x][1] for x in artuzel_articles]},
                                             'datasets': {'links': [new_tree_dict[x][0] for x in artuzel_datasets], 
                                                          'titles': [new_tree_dict[x][1] for x in artuzel_datasets]}, 
                                             'others': {'links': [new_tree_dict[x][0] for x in artuzel_others], 
                                                        'titles': [new_tree_dict[x][1] for x in artuzel_others]},
                                             'contacts': {'links': contacts_links,
                                                          'titles': contacts_titles}
                                                        }
                         }
links_and_titles

{'https://artuzel.com/': {'articles': {'links': ['http://artuzel.com/competitions',
    'http://artuzel.com/lectures',
    'http://artuzel.com/interview',
    'http://artuzel.com/articles',
    'http://artuzel.com/festivals'],
   'titles': ['Конкурсы и гранты',
    'Лекции',
    'Интервью и дискуссии',
    'Статьи',
    'Фестивали']},
  'contacts': {'links': ['', '', ''],
   'titles': ['Telegram', 'Facebook', 'Instagram']},
  'datasets': {'links': [], 'titles': []},
  'news': {'links': ['http://artuzel.com/news',
    'http://artuzel.com/exhibitions',
    'http://artuzel.com/ru/regions'],
   'titles': ['Новости', 'Выставки', 'Не столицы']},
  'others': {'links': [], 'titles': []}}}

## Dict updating

In [ ]:
sites_for_parsing = open_parsing_dict() # 4.1 Open dict with sites for parsing
sites_for_parsing.update(links_and_titles)

In [ ]:
len(sites_for_parsing)

4

In [ ]:
sites_for_parsing

{'https://artchive.ru/': {'articles': {'links': ['https://artchive.ru/publications',
    'https://artchive.ru/publications/analysis',
    'https://artchive.ru/publications/for_beginers',
    'https://artchive.ru/publications/experts',
    'https://artchive.ru/publications/collectors',
    'https://artchive.ru/publications/aritist_biographies',
    'https://artchive.ru/publications/for_artists'],
   'titles': ['Все статьи',
    'Аналитика',
    'Новичкам',
    'Экспертам',
    'Коллекционерам',
    'Персоналии',
    'Художникам']},
  'contacts': {'links': ['https://telegram.me/artchiveru',
    'https://www.facebook.com/artchive.ru',
    'https://www.instagram.com/artchiveru/'],
   'titles': ['Telegram', 'Facebook', 'Instagram']},
  'datasets': {'links': [], 'titles': []},
  'news': {'links': ['https://artchive.ru/news',
    'https://artchive.ru/news/rubric:events',
    'https://artchive.ru/news/rubric:news_about_museums_and_galleries',
    'https://artchive.ru/news/rubric:sales_and_auct

In [ ]:
save_parsing_dict(sites_for_parsing) # 3.1 Save dict with sites for parsing

# Parsing studies

### page counter

In [ ]:
sites_for_parsing = open_parsing_dict() # 4.1 Open dict with sites for parsing

parsing_links_list = [sites_for_parsing['https://artuzel.com/']['news']['links'][0]]
parsing_titles_list = [sites_for_parsing['https://artuzel.com/']['news']['titles'][0]]

print(type(parsing_links_list))
print(parsing_links_list)
print(type(parsing_titles_list))
print(parsing_titles_list)

<class 'list'>
['http://artuzel.com/news']
<class 'list'>
['Новости']


In [ ]:
parsing_links_list = []
parsing_titles_list = []

for el in sites_for_parsing['https://artuzel.com/']['news']['links']:
  parsing_links_list.append(el)
for el in sites_for_parsing['https://artuzel.com/']['news']['titles']:
  parsing_titles_list.append(el)
for el in sites_for_parsing['https://artuzel.com/']['articles']['links']:
  parsing_links_list.append(el)
for el in sites_for_parsing['https://artuzel.com/']['articles']['titles']:
  parsing_titles_list.append(el)

parsing_links_list

['http://artuzel.com/news',
 'http://artuzel.com/exhibitions',
 'http://artuzel.com/ru/regions',
 'http://artuzel.com/competitions',
 'http://artuzel.com/lectures',
 'http://artuzel.com/interview',
 'http://artuzel.com/articles',
 'http://artuzel.com/festivals']

In [ ]:
page = get_page(parsing_links_list[0])
page.title.string

'Новости | | ART Узел'

In [ ]:
# 8.3 Parsing artuzel pages_counter_stat - saving last parsing results from the first pages

def get_artuz_pages_counter_stat(page_url, page, parsing_links, parsing_titles):
  start_time = time.time()

  # counter block of the page with news list - one for every news' category
  table_blocks = page.find('li', class_='pager-current')
  digits = []
  # problems can start from here
  try:
    digits.append(int(table_blocks.get_text().split(' ')[0]))
    digits.append(int(table_blocks.get_text().split(' ')[2]))
  except:
    digits = [np.nan, np.nan]

  global df_counter_new
  df_counter_new = pd.DataFrame({'page_url': [page_url],
                                 'page_title': [parsing_titles[parsing_links.index(page_url)]],
                                 'records_total': [np.nan],
                                 'pages_total': [digits[1]],
                                 'records_max': [np.nan],
                                 'records_value': [np.nan],
                                 'first_page_url': [page_url],
                                 'parsing_date': [datetime.now().date()],
                                 'parsing_time': [datetime.now().time()],
                                 'parsing_sec_speed': [time.time() - start_time]
                                 })
  
  return df_counter_new

In [ ]:
for link in parsing_links_list:
  link_page = get_page(link) # 1. Collect request status
  print(link_page.title.string)
  get_artuz_pages_counter_stat(link, link_page, parsing_links_list, parsing_titles_list)
  df_counter = pd.concat([df_counter, df_counter_new])

Новости | | ART Узел
Выставки | | ART Узел
НЕ СТОЛИЦЫ | | ART Узел
ART Узел | Независимый медиа-проект о современном искусстве
ЛЕКЦИИ, МАСТЕР-КЛАССЫ, ТВОРЧЕСКИЕ ВСТРЕЧИ | | ART Узел
ИНТЕРВЬЮ И ДИСКУССИИ | | ART Узел
Статьи | | ART Узел
Фестивали | | ART Узел


In [ ]:
df_counter.drop_duplicates(subset = ['page_url'], keep = 'last', inplace = True)

In [ ]:
df_counter

,page_url,page_title,records_total,pages_total,records_max,records_value,first_page_url,parsing_date,parsing_time,parsing_sec_speed
0,http://artuzel.com/news,Новости,NaN,75.0,NaN,NaN,http://artuzel.com/news,2022-03-23,05:00:07.644471,0.002578
0,http://artuzel.com/exhibitions,Выставки,NaN,64.0,NaN,NaN,http://artuzel.com/exhibitions,2022-03-23,05:00:08.201159,0.001890
0,http://artuzel.com/ru/regions,Не столицы,NaN,NaN,NaN,NaN,http://artuzel.com/ru/regions,2022-03-23,05:00:08.819405,0.003159
0,http://artuzel.com/competitions,Конкурсы и гранты,NaN,33.0,NaN,NaN,http://artuzel.com/competitions,2022-03-23,05:00:09.316185,0.002196
0,http://artuzel.com/lectures,Лекции,NaN,35.0,NaN,NaN,http://artuzel.com/lectures,2022-03-23,05:00:09.899157,0.002296
0,http://artuzel.com/interview,Интервью и дискуссии,NaN,NaN,NaN,NaN,http://artuzel.com/interview,2022-03-23,05:00:10.889003,0.016619
0,http://artuzel.com/articles,Статьи,NaN,NaN,NaN,NaN,http://artuzel.com/articles,2022-03-23,05:00:12.057219,0.021557
0,http://artuzel.com/festivals,Фестивали,NaN,NaN,NaN,NaN,http://artuzel.com/festivals,2022-03-23,05:00:12.678856,0.002681


- some sections doesn't have pager counter and articles are just added to existing lists - how to collect new ???

In [ ]:
artuzel_s = '/artusel_pages_counter_stat.csv'
#save_df_file(df_counter, artuzel_s, 'live') # 2.2 Save df to csv file

In [ ]:
df_counter = open_file_csv(artuzel_s, 'live')
df_counter

,page_url,page_title,records_total,pages_total,records_max,records_value,first_page_url,parsing_date,parsing_time,parsing_sec_speed
0,http://artuzel.com/news,Новости,NaN,75.0,NaN,NaN,http://artuzel.com/news,2022-03-23,05:00:07.644471,0.002578
1,http://artuzel.com/exhibitions,Выставки,NaN,64.0,NaN,NaN,http://artuzel.com/exhibitions,2022-03-23,05:00:08.201159,0.001890
2,http://artuzel.com/ru/regions,Не столицы,NaN,NaN,NaN,NaN,http://artuzel.com/ru/regions,2022-03-23,05:00:08.819405,0.003159
3,http://artuzel.com/competitions,Конкурсы и гранты,NaN,33.0,NaN,NaN,http://artuzel.com/competitions,2022-03-23,05:00:09.316185,0.002196
4,http://artuzel.com/lectures,Лекции,NaN,35.0,NaN,NaN,http://artuzel.com/lectures,2022-03-23,05:00:09.899157,0.002296
5,http://artuzel.com/interview,Интервью и дискуссии,NaN,NaN,NaN,NaN,http://artuzel.com/interview,2022-03-23,05:00:10.889003,0.016619
6,http://artuzel.com/articles,Статьи,NaN,NaN,NaN,NaN,http://artuzel.com/articles,2022-03-23,05:00:12.057219,0.021557
7,http://artuzel.com/festivals,Фестивали,NaN,NaN,NaN,NaN,http://artuzel.com/festivals,2022-03-23,05:00:12.678856,0.002681


In [ ]:
for page in df_counter['page_url']:
  articles_page = get_page(page) # 1. Collect request status
  print(articles_page.title.string)
  items_block = articles_page.find_all('div', class_='view-content')[0].find_all('a')
  print(len(items_block))

Новости | | ART Узел
20
Выставки | | ART Узел
20
НЕ СТОЛИЦЫ | | ART Узел
45
ART Узел | Независимый медиа-проект о современном искусстве
20
ЛЕКЦИИ, МАСТЕР-КЛАССЫ, ТВОРЧЕСКИЕ ВСТРЕЧИ | | ART Узел
20
ИНТЕРВЬЮ И ДИСКУССИИ | | ART Узел
385
Статьи | | ART Узел
527
Фестивали | | ART Узел
26


In [ ]:
first_page_url = 'http://artuzel.com/news'
pages_total = 10

for p in range(1, pages_total):
  url_split = first_page_url.split('/')
  next_page_url = url_split[0]+'//'+url_split[2]+'/'+url_split[3]+'?page='+str(p)
  print(next_page_url)

http://artuzel.com/news?page=1
http://artuzel.com/news?page=2
http://artuzel.com/news?page=3
http://artuzel.com/news?page=4
http://artuzel.com/news?page=5
http://artuzel.com/news?page=6
http://artuzel.com/news?page=7
http://artuzel.com/news?page=8
http://artuzel.com/news?page=9


In [ ]:
#http://artuzel.com/news?page=1 for the second !!!!

### current data

In [ ]:
artuzel_articles = 'http://artuzel.com/exhibitions'
articles_page = get_page(artuzel_articles) # 1. Collect request status
articles_page.title.string

'Выставки | | ART Узел'

In [ ]:
items_block = articles_page.find_all('div', class_='view-content')
len(items_block)

3

In [ ]:
len(items_block[0].find_all('a'))

20

In [ ]:
for date in items_block[0].find_all('a')[:11]: # !!!!!!!!!!!!!!!!!!!!:
  # problems can start from here
  try:
    d = date.find('div', class_='views-field-field-created').get_text().split(',')[1]
  except:
    try:
      d = date.find('span', class_='date-display-single').get_text().split(',')[1]
    except: 
      datetime.now().date()
  print(d)

 15 марта 2022
 15 февраля 2022
 06 февраля 2022
 23 января 2022
 13 января 2022
 13 января 2022
 14 декабря 2021
 28 ноября 2021
 14 ноября 2021
 06 ноября 2021
 06 ноября 2021


In [ ]:
for date in items_block[0].find_all('a')[:5]:
  d = date.find('span', class_='date-display-single').get_text().split(',')[1]
  print(d.split(' '))

['', '15', 'марта', '2022']
['', '15', 'февраля', '2022']
['', '06', 'февраля', '2022']
['', '23', 'января', '2022']
['', '13', 'января', '2022']


In [ ]:
rus_mon_dict = {'янв': '01', 'фев': '02', 'мар': '03',
                'апр': '04', 'май': '05', 'июн': '06',
                'июл': '07', 'авг': '08', 'сен': '09',
                'окт': '10', 'ноя': '11', 'дек': '12'}

In [ ]:
string_1 = '01 марта 2022'
string_2 = string_1.split(' ')[0] + '.' + rus_mon_dict[string_1.split(' ')[1][:3]] + '.' + string_1.split(' ')[2]
pd.to_datetime(string_2).date()

datetime.date(2022, 1, 3)

In [ ]:
date_1 = '2022-03-01 23:43:28'
pd.to_datetime(date_1).date()

datetime.date(2022, 3, 1)

In [ ]:
# 9.4 Parsing artuzel current news_articles_data - saving last parsing results from the first pages

def get_artuz_pages_curr_data(page_url, page, parsing_links, parsing_titles):
  start_time = time.time()
  
  # News categories
  items_block = link_page.find_all('div', class_='view-content')[0].find_all('a')
  
  links_list = []
  titles_list = []
  short_text_list = []
  for item in items_block[:3]: # !!!!!!!!!!!!!!!!!!!!
    # problems can start from here
    links_list.append(item.get('href'))
    titles_list.append(item.find('div', class_='view-exhibitions-title').get_text())
    short_text_list.append(item.find('div', class_='view-exhibitions-body').get_text())
  dates_list = []
  for date in items_block[:3]: # !!!!!!!!!!!!!!!!!!!!:
    # problems can start from here
    try:
      d = date.find('div', class_='views-field-field-created').get_text().split(',')[1]
      dates_list.append(d.split(' ')[1] + '.' + rus_mon_dict[d.split(' ')[2][:3]] + '.' + d.split(' ')[3])
    except:
      try:
        d = date.find('span', class_='date-display-single').get_text().split(',')[1]
        dates_list.append(d.split(' ')[1] + '.' + rus_mon_dict[d.split(' ')[2][:3]] + '.' + d.split(' ')[3])
      except: 
        dates_list.append(datetime.now().date())

  global df_data_new
  df_data_new = pd.DataFrame({'date': dates_list, # date of news
                              'link': links_list, # link to news
                              'title': titles_list, # title of news
                              'short_text': short_text_list # news' preview
                              })
  df_data_new['page_url'] = page_url # link to category of news - the same
  df_data_new['page_title'] = parsing_titles[parsing_links.index(page_url)] # title of news' category - the same

  
  # For parsing long texts - beginning
  df_data_new['long_text_url'] = df_data_new['page_url'].apply(lambda x: x.split('/')[0] + '//' + x.split('/')[2]) + df_data_new['link']
  global long_text_url_list_new
  long_text_url_list_new = df_data_new['long_text_url'].tolist()

  df_data_new['parsing_date'] = datetime.now().date()
  df_data_new['parsing_time'] = datetime.now().time()
  df_data_new['parsing_sec_speed'] = time.time() - start_time # without long_text collection

  collected_pages_a_count = df_data_new['link'].count()

  return df_data_new, long_text_url_list_new, print('{} articles have been collected\nin {} seconds'.format(collected_pages_a_count, (time.time() - start_time)))

In [ ]:
link = df_counter['page_url'].tolist()[0]
title = df_counter['page_title'].tolist()[0]
parsing_links_list = df_counter['page_url'].tolist()
parsing_titles_list = df_counter['page_title'].tolist()

print(len(parsing_links_list))
print(link)
print(len(parsing_titles_list))
print(title)

8
http://artuzel.com/news
8
Новости


In [ ]:
link_page = get_page(link) # 1. Collect request status
print(link_page.title.string)
get_artuz_pages_curr_data(link, link_page, link, title)

Новости | | ART Узел
3 articles have been collected
in 0.007687568664550781 seconds


(         date                                               link  \
 0  18.03.2022  /content/s-13-po-17-aprelya-v-gostinom-dvore-y...   
 1  09.03.2022  /content/obyavlen-short-list-yubileynoy-x-prem...   
 2  25.02.2022             /content/skonchalsya-german-vinogradov   
 
                                                title  \
 0  С 13 по 17 апреля в Гостином Дворе ярмарка «АР...   
 1  Объявлен шорт-лист юбилейной X Премии The Art ...   
 2                        Скончался Герман Виноградов   
 
                                           short_text                 page_url  \
 0   Крупнейшая и старейшая художественная ярмарка...  http://artuzel.com/news   
 1   Победители выбираются в пяти номинациях: «Выс...  http://artuzel.com/news   
 2   Сегодня, 25 февраля 2022 года, в возрасте 64 ...  http://artuzel.com/news   
 
   page_title                                      long_text_url parsing_date  \
 0          Н  http://artuzel.com/content/s-13-po-17-aprelya-...   2022-03-23   


In [ ]:
for link in parsing_links_list:
  link_page = get_page(link) # 1. Collect request status
  print(link_page.title.string)
  get_artuz_pages_curr_data(link, link_page, parsing_links_list, parsing_titles_list)
  df_data = pd.concat([df_data, df_data_new])

Новости | | ART Узел
3 articles have been collected
in 0.011875629425048828 seconds
Выставки | | ART Узел
3 articles have been collected
in 0.007368326187133789 seconds
НЕ СТОЛИЦЫ | | ART Узел
3 articles have been collected
in 0.009479284286499023 seconds
ART Узел | Независимый медиа-проект о современном искусстве
3 articles have been collected
in 0.01019144058227539 seconds
ЛЕКЦИИ, МАСТЕР-КЛАССЫ, ТВОРЧЕСКИЕ ВСТРЕЧИ | | ART Узел
3 articles have been collected
in 0.011265039443969727 seconds
ИНТЕРВЬЮ И ДИСКУССИИ | | ART Узел
3 articles have been collected
in 0.03619742393493652 seconds
Статьи | | ART Узел
3 articles have been collected
in 0.04996967315673828 seconds
Фестивали | | ART Узел
3 articles have been collected
in 0.010009288787841797 seconds


In [ ]:
df_data.drop_duplicates(subset = 'long_text_url', keep = 'last', inplace = True)
df_data

,date,link,title,short_text,page_url,page_title,long_text_url,parsing_date,parsing_time,parsing_sec_speed
0,18.03.2022,/content/s-13-po-17-aprelya-v-gostinom-dvore-y...,С 13 по 17 апреля в Гостином Дворе ярмарка «АР...,Крупнейшая и старейшая художественная ярмарка...,http://artuzel.com/news,Новости,http://artuzel.com/content/s-13-po-17-aprelya-...,2022-03-23,09:37:33.617540,0.009897
1,09.03.2022,/content/obyavlen-short-list-yubileynoy-x-prem...,Объявлен шорт-лист юбилейной X Премии The Art ...,Победители выбираются в пяти номинациях: «Выс...,http://artuzel.com/news,Новости,http://artuzel.com/content/obyavlen-short-list...,2022-03-23,09:37:33.617540,0.009897
2,25.02.2022,/content/skonchalsya-german-vinogradov,Скончался Герман Виноградов,"Сегодня, 25 февраля 2022 года, в возрасте 64 ...",http://artuzel.com/news,Новости,http://artuzel.com/content/skonchalsya-german-...,2022-03-23,09:37:33.617540,0.009897
0,15.03.2022,/content/sobytiya-14-22-marta,события 14-22 марта,Подготовили для вас небольшую подборку терапев...,http://artuzel.com/exhibitions,Выставки,http://artuzel.com/content/sobytiya-14-22-marta,2022-03-23,09:37:34.212532,0.006955
1,15.02.2022,/content/sobytiya-21-27-fevralya,события 21-27 февраля,Зовем вас на новые выставки в Москве и Петербу...,http://artuzel.com/exhibitions,Выставки,http://artuzel.com/content/sobytiya-21-27-fevr...,2022-03-23,09:37:34.212532,0.006955
2,06.02.2022,/content/sobytiya-7-13-fevralya,события 7-13 февраля,Подготовили для вас новую подборку событий в М...,http://artuzel.com/exhibitions,Выставки,http://artuzel.com/content/sobytiya-7-13-fevralya,2022-03-23,09:37:34.212532,0.006955
0,22.03.2022,/content/novye-vystavki-v-galeree-viktoriya-v-...,Новые выставки в галерее «Виктория» в Самаре,"С 31 марта по 21 мая выставка ""После фотограф...",http://artuzel.com/ru/regions,Не столицы,http://artuzel.com/content/novye-vystavki-v-ga...,2022-03-23,09:37:34.856531,0.009084
1,20.03.2022,/content/raboty-vladimira-lagranzha-i-aleksand...,Работы Владимира Лагранжа и Александра Родченк...,С 18 марта по 22 мая в Арт-галерее Ельцин Цент...,http://artuzel.com/ru/regions,Не столицы,http://artuzel.com/content/raboty-vladimira-la...,2022-03-23,09:37:34.856531,0.009084
2,02.03.2022,/content/art-praktiki-dlya-vseh-vozrastov-v-mc...,Арт-практики для всех возрастов в МЦ «Площадь ...,Музейный центр «Площадь Мира» запускает три но...,http://artuzel.com/ru/regions,Не столицы,http://artuzel.com/content/art-praktiki-dlya-v...,2022-03-23,09:37:34.856531,0.009084
0,20.03.2022,/content/open-call-obedineniya-all-one-one-all...,Open call Объединения ALL FOR ONE ONE FOR ALL ...,Время объединяться.Все четыре года существован...,http://artuzel.com/competitions,Конкурсы и гранты,http://artuzel.com/content/open-call-obedineni...,2022-03-23,09:37:35.381153,0.009615


In [ ]:
for url in df_data_new['long_text_url']:
  print(url)

http://artuzel.com/content/s-13-po-17-aprelya-v-gostinom-dvore-yarmarka-art-moskva
http://artuzel.com/content/obyavlen-short-list-yubileynoy-x-premii-art-newspaper-russia
http://artuzel.com/content/skonchalsya-german-vinogradov


In [ ]:
#df_data = df_data_new.copy()

In [ ]:
#df_data = pd.concat([df_data, df_data_new])

In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 2
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               24 non-null     object 
 1   link               24 non-null     object 
 2   title              24 non-null     object 
 3   short_text         24 non-null     object 
 4   page_url           24 non-null     object 
 5   page_title         24 non-null     object 
 6   long_text_url      24 non-null     object 
 7   parsing_date       24 non-null     object 
 8   parsing_time       24 non-null     object 
 9   parsing_sec_speed  24 non-null     float64
dtypes: float64(1), object(9)
memory usage: 2.1+ KB


In [ ]:
artuzel_d = '/artuzel_news_articles_data.csv'
save_df_file(df_data, artuzel_d, 'live') # 2.2 Save df to csv file

### long text

In [ ]:
artuzel_d = '/artuzel_news_articles_data.csv'
df_data = open_file_csv(artuzel_d, 'live')

In [ ]:
long_text_url_list_curr = df_data['long_text_url'].tolist()
len(long_text_url_list_curr)

24

In [ ]:
lt_url_list = long_text_url_list_curr[::3]
lt_url_list

['http://artuzel.com/content/s-13-po-17-aprelya-v-gostinom-dvore-yarmarka-art-moskva',
 'http://artuzel.com/content/sobytiya-14-22-marta',
 'http://artuzel.com/content/novye-vystavki-v-galeree-viktoriya-v-samare',
 'http://artuzel.com/content/open-call-obedineniya-all-one-one-all-do-25-marta',
 'http://artuzel.com/content/18-i-20-marta-meropriyatiya-v-3120-gallery',
 'http://artuzel.com/content/yaroslav-kudryashov-liminalnost-ravna-odinochestvu',
 'http://artuzel.com/content/vospominaniya-o-poezdke-v-nikola-lenivec',
 'http://artuzel.com/content/mezhdunarodnyy-festival-mediaiskusstva-kiberfest-13-v-sankt-peterburge-s-11-po-28-noyabrya']

In [ ]:
lt_url = lt_url_list[7]
text_page = get_page(lt_url) # 1. Collect request status
text_page.title.string

'Международный фестиваль медиаискусства «Киберфест-13» в Санкт-Петербурге с 11 по 28 ноября | ART Узел'

In [ ]:
len(text_page.find('div', class_='content clearfix').find_all('span'))

4

In [ ]:
for sp in text_page.find('div', class_='content clearfix').find_all('span'):
  print(sp.text)

William Latham (UK) & Mutator VR Team, Fantasy Virus (Mutator), custom computer-generated VR Art Video, 2020–2021 © William Latham
Lingdong Huang (China–USA) & Ziwei Wu (China–UK), Mimicry, video installation, 2020, exhibition view at “Chimera Garden”, Goldsmiths, London, 2020 © Lingdong Huang & Ziwei Wu
Rebecca Merlic (Europe), TheCityAsAHouse, interactive new media digital software art, 2020 © Rebecca Merlic
11–28 ноября 2021


In [ ]:
for em in text_page.find('div', class_='content clearfix').find_all('em'):
  print(em)

<em>Ирина Затуловская</em>
<em>Федор Телков. Уставщик Поморской Челябинской общины возле памятного креста у поморского старообрядческого кладбища. Таватуй, 2021</em>
<em>Максим Ксута. Из серии «Осколки памяти», 2017</em>
<em>Мраморизированный доломит, резьба</em>
<em>Риёти Курокава (Япония), elementum, 2018.</em>
<em>Фотография: Takuro Someya Contemporary Art</em>
<em><img alt="" class="media-image" height="480" src="http://artuzel.com/sites/default/files/styles/large/public/aggregation17-de095._smeshannaya_tehnika_s_ispolzovaniem_koreyskoy_shelkovichnoy_bumagi_163x228_sm._2017_g._small_1.jpg?itok=toeR9A-I" style="display: block; margin-left: auto; margin-right: auto;" width="674"/></em>
<em>Aggregation17-DE095. смешанная техника с использованием корейской шелковичной бумаги, 163x228 см. 2017 г.</em>


In [ ]:
# 10.4 Parsing artuzel long texts - saving only new results from the first pages
    
def get_artuz_long_text(text_page):
  start_time = time.time()

  t_block = text_page.find('div', class_='content clearfix').find_all('p')
  img_block = text_page.find('div', class_='content clearfix').find_all('img')
  tag_block = text_page.find('ul', class_='links inline')

  # different code
  global tags_list_new
  tags_list_new = []
  try:
    for tag in tag_block.find_all('a'):
      tags_list_new.append(tag.get('href'))
      tags_list_new.append(tag.get_text())
  except: pass

  global long_text_list_new
  long_text_list_new = []
  global headings_list_new
  headings_list_new = []
  # date of the news - for replacing in current data
  # new part
  d = text_page.find('div', class_='meta submitted').get_text().split(',')[1]
  headings_list_new.append(d.split(' ')[1] + '.' + rus_mon_dict[d.split(' ')[2][:3]] + '.' + d.split(' ')[3])
  
  # 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'div' class_='postListItemTitleRoot', 'b', 'blockquote'
  try:
    long_text_list_new.append(text_page.find('h1', id='page-title').get_text())
    headings_list_new.append(text_page.find('h1', id='page-title').get_text())
    for t in t_block:
      long_text_list_new.append(t.get_text())
      for h in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
        h_block = t.find_all(h)
        try:
          for h_str in h_block:
            headings_list_new.append(h_str.get_text())
        except: continue
      for b in ['strong', 'span', 'em', 'b']:
        b_block = t.find_all(b)
        try:
          for b_str in b_block:
            if b_str not in headings_list_new:
              headings_list_new.append(b_str.get_text())
        except: continue

  except: pass

  global images_list_new
  images_list_new = []
  # title can be not full
  try:
    for img in img_block:
      images_list_new.append(img.get('src'))
      try:
        images_list_new.append(img.get('title'))
      except:
        images_list_new.append('')
  except: pass

  global sourses_list_new
  sourses_list_new = []
  global parsing_speed_list_new
  parsing_speed_list_new = [(time.time() - start_time)]

  return long_text_list_new, headings_list_new, images_list_new, sourses_list_new, tags_list_new, parsing_speed_list_new

In [ ]:
long_text_list = []
headings_list = []
images_list = []
sourses_list = []
tags_list = []
parsing_speed_list = []

for url in long_text_url_list_curr: # one article from certane category news list
  page = get_page(url) # 1. Collect request status

  get_artuz_long_text(page)

  long_text_list.append(long_text_list_new)
  headings_list.append(headings_list_new)
  images_list.append(images_list_new)
  sourses_list.append(sourses_list_new)
  tags_list.append(tags_list_new)
  parsing_speed_list.append(parsing_speed_list_new)

#### loop ending

long_text_df_new = pd.DataFrame({'long_text_url': long_text_url_list_curr,
                                 'long_text': long_text_list,
                                 'structure': headings_list,
                                 'images': images_list,
                                 'sourses': sourses_list,
                                 'tags': tags_list,
                                 'parsing_sec_speed_2': parsing_speed_list # for long_text collection without getting page
                                 })
  
collected_pages_t_count = long_text_df_new['long_text_url'].count()

print('{} text pages have been collected\nAll text pages have been collected - {}'.format(collected_pages_t_count, collected_pages_t_count==len(long_text_url_list_curr)))

24 text pages have been collected
All text pages have been collected - True


In [ ]:
long_text_df_new.tail()

,long_text_url,long_text,structure,images,sourses,tags,parsing_sec_speed_2
19,http://artuzel.com/content/telo-kak-metafora-n...,[\n Тело как метафора на выставке «Ко...,"[16.03.2022, \n Тело как метафора на ...",[http://artuzel.com/sites/default/files/styles...,[],"[/tegi/natalya-drachinskaya, Наталья Драчинская]",[0.007515907287597656]
20,http://artuzel.com/content/tush-v-temnote-vrem...,[\n «Тушь в темноте» временно мимикри...,"[16.03.2022, \n «Тушь в темноте» врем...",[http://artuzel.com/sites/default/files/styles...,[],"[/tegi/katya-ulitina, Катя Улитина, /tegi/niko...",[0.008871316909790039]
21,http://artuzel.com/content/mezhdunarodnyy-fest...,[\n Международный фестиваль медиаиску...,"[03.11.2021, \n Международный фестива...",[http://artuzel.com/sites/default/files/styles...,[],[],[0.00823068618774414]
22,http://artuzel.com/content/s-15-po-21-noyabrya...,[\n С 15 по 21 ноября в Красноярске п...,"[07.10.2021, \n С 15 по 21 ноября в К...",[http://artuzel.com/sites/default/files/styles...,[],[],[0.0075566768646240234]
23,http://artuzel.com/content/s-23-po-26-sentyabr...,[\n С 23 по 26 сентября в Петербурге ...,"[22.09.2021, \n С 23 по 26 сентября в...",[http://artuzel.com/sites/default/files/styles...,[],[],[0.006603240966796875]


In [ ]:
for line in long_text_df_new['structure']:
  print(len(line), line)

5 ['18.03.2022', '\n          С 13 по 17 апреля в Гостином Дворе ярмарка «АРТ МОСКВА»        ', 'РАЗДЕЛ «СОВРЕМЕННОЕ ИСКУССТВО»', 'Édition studio (HK), Jeff Koons (after) Balloon Dog Cold cast resin', 'РАЗДЕЛ «КЛАССИЧЕСКОЕ ИСКУССТВО»']
30 ['15.03.2022', '\n          события 14-22 марта        ', 'Охота на искусство', 'до 18 марта', 'Зверевский центр современного искусства', '«Мумии древнего Египта. Искусство бессмертия» и «Ирина Затуловская. Бегство в Египет» ', 'до 31 мая', 'ГМИИ им. А.С. Пушкина', 'Ирина Затуловская', 'Федор Телков "Право веры"', '22 марта — 8 мая', 'ММОМА', 'Федор Телков. Уставщик Поморской Челябинской общины возле памятного креста у поморского старообрядческого кладбища. Таватуй, 2021', 'Максим Ксута АРХИТЕКТОНИКА', 'до 20 марта', 'Триумф', 'Максим Ксута. Из серии «Осколки памяти», 2017', 'Мраморизированный доломит, резьба', 'выставка «New elements»', 'до 27 марта', 'Западное крыло Новой Третьяковки', 'Риёти Курокава (Япония), elementum, 2018.', 'Риёти Курокава (Яп

In [ ]:
artuzel_lt = '/artuzel_long_text.csv'
#save_df_file(long_text_df_new, artuzel_lt, 'live') # 3. Save df to csv file

In [ ]:
long_text_df = open_file_csv(artuzel_lt, 'live')

In [ ]:
for lane in long_text_df['structure'].sample(10):
  print(lane[2:-2].split("', '")[0], type(lane[2:-2].split("', '")[0]))

13.01.2022 <class 'str'>
21.12.2021 <class 'str'>
14.12.2021 <class 'str'>
14.06.2021 <class 'str'>
08.07.2021 <class 'str'>
21.10.2021 <class 'str'>
02.12.2021 <class 'str'>
09.10.2021 <class 'str'>
09.12.2021 <class 'str'>
16.03.2022 <class 'str'>
